# Header

In [1]:
#Loading Hyperspy Utilities and additional libraries
%matplotlib qt5
import numpy as np
import hyperspy
import hyperspy.api as hs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
import matplotlib.image as mpimg
from tqdm import tqdm, trange, tnrange, tqdm_notebook
import cv2
import sys
import os
from sympy import Symbol
from joblib import Parallel, delayed
import multiprocessing
hs.preferences.gui(toolkit='ipywidgets')

#Loading libraries for watershed-analysis
import mendeleev as mend
import sympy
import scipy
from scipy.signal import argrelextrema
from scipy import ndimage as ndi
from scipy.optimize import curve_fit
from skimage.filters import rank
from skimage.morphology import (
    watershed, disk, diamond, binary_erosion, binary_dilation)
from skimage.util import img_as_ubyte

/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


# Mapping automation _Plasmon-characteristics_ 

## Class _Plasmon feature Mapping_

In [4]:
# Class shall be used to map shear-transformation-zones for an image to visualize 
# shear-bands by considering energy-shifts of the plasmon-peaks aligned with the ZLP
class Plasmon_mapper(object): 

    # Initialize the class for STZ-mapping 
    def __init__(self, 
                 filename      = '', 
                 file          = None,
                 is_stack      = False, 
                 is_lazy       = False, 
                 binning       = True, 
                 includes_fits = False,
                 deconv        = False
                ):
        """
        Plasmon_mapper: 
                    With careful analysis of an EELS-image the change of the spatial electron density
                    and additional thickness analysis, the MSER algorithm generates segments of the 
                    sheared regions and evaluates orientation of sheared regions, as the strain
                    impacts on the Plasmon resonance energy.
                    Different outputs are possible to analyze the parameter shifts of the plasmon 
                    signature in the specimen.
                    
                    The minimum size of the spectrum image/list supported by this class is: 
                    (1-dim: 10 | 2-dim: 10*10=100) 
        ----------------------------------------------------------------------------------------------
        Initialization parameters:
                    filename: STRING      - specify filename and directory to load the EELS-image generated 
                                            (e.g. in GATAN)
                    file: Hyperspy-Signal - only use this when not using filename. Previously initia-
                                            lized Hyperspy Signals object. Read more on the Signals 
                                            objects http://hyperspy.org/hyperspy-doc/
                    is_stack: BOOLEAN     - loading multiple files by a wildcard "*"
                    
                    is_lazy: BOOLEAN      - to analyze big data (e.g. .dm4 - files) can be a problematic
                                            thing to due high memory requirements, therefore instead of
                                            using a numpy nd.array one can fall back to dask.dask_array
                                            resulting in an object which consists of multiple numpy arrays.
                                            This makes it possible to only work on parts of the datafile
                                            and therefore requiring only a small part of the memory require-
                                            ments.
                    binning: BOOLEAN      - operate on binned datasets. Due to caution set to FALSE as 
                                            default! Preferably, the datasets should only be binned using
                                            hyperspy to verify correct results.
        """
        self.Filename          = ''
        self.File              = None
        
        self.File_deconv       = None
        self.attr_deconv       = deconv
        
        self.Darkfield         = None
        
        self.Fit_Model         = None
        self.models_dict       = { '0' : 'VolumePlasmonDrude_leastsq_ls',
                                   '1' : 'VolumePlasmonDrude_mpfit_ls',
                                   '2' : 'VolumePlasmonDrude_NelderMead_ml',
                                   '3' : 'Lorentzian_leastsq_ls',
                                   '4' : 'Lorentzian_mpfit_ls',
                                   '5' : 'Lorentzian_NelderMead_ml',
                                   '6' : 'Gaussian_leastsq_ls',
                                   '7' : 'Gaussian_mpfit_ls',
                                   '8' : 'Gaussian_NelderMead_ml',
                                   '9' : 'Voigt_leastsq_ls',
                                   '10' : 'Voigt_mpfit_ls',
                                   '11' : 'Voigt_NelderMead_ml'
                                 }
        
        self.Chisq             = None
        self.red_Chisq         = None
        
        self.rchisq_mean       = None
        self.rchisq_std        = None
        
        self.eels_axis         = None
        self.function_set      = None
        self.model_name        = None
        
        self.ZLP_Emax          = None
        self.ZLP_FWHM          = None
        self.ZLP_Int           = None
        
        self.FPP_Emax          = None
        self.FPP_FWHM          = None
        self.FPP_Int           = None
        self.SPP_Emax          = None
        self.SPP_FWHM          = None
        self.SPP_Int           = None
        
        self.Ep_q0             = None
        
        self.param_dict        = {}
        self.param_dict_deconv = {}
        
        self.line_scans        = {}

        self.Elements          = None
        self.Concentrations    = None
        self.thickness_map     = None
        
        self.dir_list          = ['Parameter Maps', 
                                  'Thickness Evaluation', 
                                  'Density Analysis', 
                                  'Plasmon characteristics'
                                 ]
        
        self.roi               = None
        self.roi_Ep            = None
        self.line              = None
        
        self.elastic_threshold = None # arithmetic mean of the elastic threshold over the navigation space
        self.elastic_intensity = None
        
        self.load_data(filename, 
                       file, 
                       is_stack, 
                       is_lazy, 
                       binning
                      )
        
    
    def print_file_information(self):
        """
        Printing standard metadata of file for beam and detector information as well as dataset information
        and the currently stored models of the file
        """
        print(self.File.metadata)
        
        print(self.File.axes_manager)
        
        print(self.File.models)
        
        
    def yes_or_no(self, question):
        """
        Function for yes-no user input. Returns BOOLEAN value (y = True | n = False)
        """
        reply = str(input(question+' (y/n): ')).lower().strip()
        
        if (reply == ''):
            return self.yes_or_no("Pleas Enter")
        
        elif (reply[0] == 'y'):
            return True
        
        elif (reply[0] == 'n'):
            return False
        
        else:
            return self.yes_or_no("Please Enter")
        
    
    def detector_parameter_gui(self):
        """
        Comments missing
        """
        self.File.set_microscope_parameters()
    
    
    def model_gui(self, model):
        """
        Function to call the model attribute gui and await parameter adjustments until user is finished
        """
        model.gui()
        while True:
            finished = self.yes_or_no('When finished type (y) :')
            if (finished == True):
                break
        
    
    def split(self, txt, seps):
        """
        Splitting a string by specified seperators seps taking a tuple as input.
        """
        default_sep = seps[0]

        # we skip seps[0] because that's the default seperator
        for sep in seps[1:]:
            txt = txt.replace(sep, default_sep)
        return [i.strip() for i in txt.split(default_sep)]

        
    # Routine to load the data
    def load_data(self, 
                  filename, 
                  file, 
                  is_stack, 
                  is_lazy,
                  binning
                 ):
        """
        load_data: This routine specifies the alignment of the space dimensions and
                   the zero-loss alignment as well as poissonian noise estimation and
                   optional model loading. Also it will consider a deconvolution by
                   the fourier log method (fully automated - still needs testing).
        ----------------------------------------------------------------------------------------------
        Initialization parameters:
                    filename: STRING - specify filename and directory to load the EELS-image generated 
                                       (e.g. in GATAN)
                    file: Hyperspy-Signal - only use this when not using filename. Previously initia-
                                            lized Hyperspy Signals object. Read more on the Signals 
                                            objects http://hyperspy.org/hyperspy-doc/
                    is_stack: BOOLEAN - loading multiple files by a wildcard "*"
                    
                    is_lazy: BOOLEAN - to analyze big data (e.g. .dm4 - files) can be a problematic
                                       thing to due high memory requirements, therefore instead of
                                       using a numpy nd.array one can fall back to dask.dask_array
                                       resulting in an object which consists of multiple numpy arrays.
                                       This makes it possible to only work on parts of the datafile
                                       and therefore requiring only a small part of the memory require-
                                       ments.
                                       
                    binning: BOOLEAN - should always be set to TRUE for EELS analysis
                                       as recommended by hyperspy's documentation.
                                       
                    deconv: BOOLEAN - specifies usage of the fourier log deconvolution
                                      to estimate the single scattered spectrum, isolating
                                      the plasmon peak without additional convolution.
                                      This can lead to better results, depending on the 
                                      quality of the SI and robustness of the fourier log
                                      method (worse results also possible).
        """
        if (filename  != ''):
            self.File  = hs.load(filename, stack = is_stack, lazy = is_lazy)

        elif (file    != None):
            self.File  = file

        self.Filename          = self.File.metadata.General.title
        
        axes                   = self.File.axes_manager.as_dictionary()
        for i in range(len(axes)):
            axis = axes['axis-' + str(i)]
            if (axis['units'] == 'eV'):
                self.eels_axis = self.File.axes_manager[i]

        including_fits = self.yes_or_no('Does the file include stored models from previous fitting?'
                                       )
        if (including_fits   == True):
            self.load_model()
        
        elif (including_fits == False):
            channels_before                     = self.File.axes_manager.signal_size
            self.File.data[self.File.data <= 0] = 1
            
            print('Setting up proper navigation space...')
            self.align_dataset()
            
            print('Aligning Zero-Loss Peak...')
            self.File.align_zero_loss_peak(subpixel = True)
            
            print('Aligning datastructure successful. Estimate poissonian noise...')
            self.File.estimate_poissonian_noise_variance()
            
            if (self.attr_deconv == True):
                print('Calculating deconvoluted spectrum...')
                self.zlp_deconvolution()
            
            channels_after   = self.File.axes_manager.signal_size
            cropped_channels = channels_before - channels_after
            print('Loading process completed. Energy channels cropped by ZLP-Alignment: ' + 
                  str(cropped_channels) + '[' + str(cropped_channels/channels_before*100) + '%]' +
                  '\n' + 'Channels before: ' + str(channels_before) + '\n' +
                  'Channels after: ' + str(channels_after)
                 )

        
        self.elastic_threshold = np.mean(
            self.File.estimate_elastic_scattering_threshold().data
        )
        self.elastic_intensity = np.mean(
            self.File.estimate_elastic_scattering_intensity(self.elastic_threshold).data
        )
        self.File.metadata.Signal.binned = binning
        
        print('Please check microscope and detector parameters,\n' + 
              'as the metadata from gatan might be incorrectly loaded\n' +
              'Hyperspy.'
             )
        
        darkfield = self.yes_or_no('Do you want to load the corresponding darkfield image?\n' +
                                   '(Warning: The dimensions have to match exactly for some functions!)'
                                  )
        if (darkfield == True):
            filename = str(input('Enter filename (has to be in the same folder as the SI): '))
            self.Darkfield = hs.load(filename)
        
        self.detector_parameter_gui()
        
    
    def align_dataset(self):
        """
        align_dataset: By dimension the dataset will be devided into different objects as follows:
                           EELS: 1D array specifying Counts/Intensity along the energy loss axis
                                 given by offset and gain as defined in the metadata
                           EELS-stack: 2D array specifying Counts/Intensity as above (see Object EELS)
                                       and 1 navigation axis
                           EELS-image: 3D array specifying Counts/Intensity as above (see Object EELS)
                                       and 2 navigation axis - therefore the name... :)
        ----------------------------------------------------------------------------------------------
        Initialization parameters:
                    None, all parameters should have been included in the metadata by GATAN or by
                    defining the meta information in hyperspy yourself.
        """
        self.File = self.File.as_signal1D(spectral_axis=self.eels_axis)
        
        self.File.set_signal_type("EELS")
        self.File.axes_manager.signal_axes[0].name = 'Energy loss'
        
        for axis in self.File.axes_manager.navigation_axes:
            axis.offset = 0
            axis.scale  = abs(axis.scale)

    
    def zlp_deconvolution(self):
        """
        Uses Fourier-Log method for deconvolution and elastic scattering threshold to estimate ZLP.
        """
        zlp              = self.File.isig[:self.elastic_threshold]
        self.File_deconv = self.File.fourier_log_deconvolution(zlp)
        
    
    def init_func(self):
        """
        Initialize distribution type for ZLP, FPP, SPP. Three types are currently supported:
        func: {'VolumePlasmonDrude', 'Lorentzian', 'Gaussian', 'Voigt'}
        """
        param_init = self.init_model_params()
        
        zlp_pos    = param_init[0]
        zlp_fwhm   = param_init[1]
        zlp_int    = param_init[2]
        fpp_pos    = param_init[3]
        fpp_fwhm   = param_init[4]
        fpp_int    = param_init[5]
        spp_pos    = param_init[6]
        spp_fwhm   = param_init[7]
        spp_int    = param_init[8]
        
        try:
            if (self.function_set == 'VolumePlasmonDrude'):

                func_1 = hs.model.components1D.Voigt()
                func_1.area.value                = zlp_int
                func_1.centre.value              = zlp_pos
                func_1.FWHM.value                = zlp_fwhm / 2
                func_1.gamma.value               = zlp_fwhm / 2
                func_1.name = 'Zero_Loss_Peak'
                
                func_2 = hs.model.components1D.VolumePlasmonDrude(intensity      = fpp_int,
                                                                  plasmon_energy = fpp_pos,
                                                                  fwhm           = fpp_fwhm
                                                                 )
                func_2.name = 'First_Plasmon_Peak'

                func_3 = hs.model.components1D.VolumePlasmonDrude(intensity      = spp_int,
                                                                  plasmon_energy = spp_pos,
                                                                  fwhm           = spp_fwhm
                                                                 )
                func_3.name = 'Second_Plasmon_Peak'
                
            elif (self.function_set == 'Lorentzian'):

                func_1 = hs.model.components1D.Voigt()
                func_1.area.value                = zlp_int
                func_1.centre.value              = zlp_pos
                func_1.FWHM.value                = zlp_fwhm / 2
                func_1.gamma.value               = zlp_fwhm / 2
                func_1.name = 'Zero_Loss_Peak'
                
                func_2 = hs.model.components1D.Lorentzian(A      = fpp_int, 
                                                          centre = fpp_pos,
                                                          gamma  = fpp_fwhm / 2
                                                         )
                func_2.name = 'First_Plasmon_Peak'

                func_3 = hs.model.components1D.Lorentzian(A      = spp_int, 
                                                          centre = spp_pos,
                                                          gamma  = spp_fwhm / 2
                                                         )
                func_3.name = 'Second_Plasmon_Peak'
                
            elif (self.function_set == 'Gaussian'):

                func_1 = hs.model.components1D.Voigt()
                func_1.area.value                = zlp_int
                func_1.centre.value              = zlp_pos
                func_1.FWHM.value                = zlp_fwhm / 2
                func_1.gamma.value               = zlp_fwhm / 2
                func_1.name = 'Zero_Loss_Peak'

                func_2 = hs.model.components1D.Gaussian(A      = fpp_int,
                                                        centre = fpp_pos,
                                                        sigma  = fpp_fwhm / (2 * np.sqrt( np.log(2) ))
                                                       )
                func_2.name = 'First_Plasmon_Peak'

                func_3 = hs.model.components1D.Gaussian(A      = spp_int,
                                                        centre = spp_pos,
                                                        sigma  = spp_fwhm / (2 * np.sqrt( np.log(2) ))
                                                       )
                func_3.name = 'Second_Plasmon_Peak'

            elif (self.function_set == 'Voigt'):

                func_1 = hs.model.components1D.Voigt()
                func_1.area.value                = zlp_int
                func_1.centre.value              = zlp_pos
                func_1.FWHM.value                = zlp_fwhm / 2
                func_1.gamma.value               = zlp_fwhm / 2
                func_1.name = 'Zero_Loss_Peak'
                
                func_2 = hs.model.components1D.Voigt()
                func_2.area.value   = fpp_int
                func_2.centre.value = fpp_pos
                func_2.FWHM.value   = fpp_fwhm / 2
                func_2.gamma.value  = fpp_fwhm / 2
                func_2.name = 'First_Plasmon_Peak'
                
                func_3 = hs.model.components1D.Voigt()
                func_2.area.value   = spp_int
                func_2.centre.value = spp_pos
                func_2.FWHM.value   = spp_fwhm / 2
                func_2.gamma.value  = spp_fwhm / 2
                func_3.name = 'Second_Plasmon_Peak'
            
            return func_1, func_2, func_3
            
        except:
            print('No distributions initialized. Please try again. For more information, see docstring.')
        

    
    # Function to fit EELS-Spectra with three gaussian
    # Another option is to use code in 2nd cell to use extra class
    def eels_fit_routine(self, 
                         function_set='VolumePlasmonDrude', 
                         fitter='leastsq', 
                         method='ls', 
                         samfire=False, 
                         multithreading=False, 
                         workers=4
                        ):         
        """
        Routine to initialize fit routine. Storing the calculated model in the loaded file.
        
        Initialize distribution model for ZLP, FPP, SPP. Three types are currently supported:
        function_Set: {'VolumePlasmonDrude', 'Lorentzian', 'Gaussian', 'Voigt'}
        """
        # Initialize serial fitting routine
        self.function_set = function_set
        self.model_name = function_set + str('_') + fitter + str('_') + method
        
        if (samfire == False):

            self.Fit_Model = self.fit_eels(fitter, 
                                           method
                                          )

        # Initialize SAMFire (parallel) fitting routine
        else:

            self.Fit_Model = self.fit_eels_SAMF(fitter, 
                                                method, 
                                                multithreading, 
                                                workers
                                               )
        
        print('Model will be stored in file...')
        self.Fit_Model.store(name = self.model_name)
        
        print('Stored models in file:')
        if (self.attr_deconv == False):
            print(self.File.models)

        elif (self.attr_deconv == True):
            print(self.File_deconv.models)
            
        self.generate_param_maps(method)
    
    
    def init_model_params(self):
        """
        Comments missing
        """        
        print('Estimate function parameters...')
        
        mean            = self.File.mean()

        # estimate peak parameters for the zero loss peak
        elastic         = mean.isig[:a.elastic_threshold]

        # as ohaver implementation only looks for peaks in the positive energy 
        # range, the axis will be shifted forward for this calculation.
        # the shift will be considered for the peak position afterwards,
        # as it behaves linear.
        offset          = elastic.axes_manager.signal_axes[0].offset
        elastic.axes_manager.signal_axes[0].offset -= offset

        axis_elastic    = elastic.axes_manager.signal_axes[0]
        amp_zlp         = np.max(elastic, axis=axis_elastic).data[0]
        param_elastic   = elastic.find_peaks1D_ohaver(amp_thresh = 0.1*amp_zlp,
                                                      maxpeakn   = 1
                                                     )
        
        if (len(param_elastic) == 1):
            zlp_pos     = param_elastic[0][0][0] + offset
            zlp_fwhm    = param_elastic[0][0][2]
            
            zlp_int     = self.elastic_intensity

        else:
            zlp_pos     = 0. # in eV
            zlp_fwhm    = 1. # in eV
            zlp_int     = self.elastic_intensity

        # estimate peak parameters for the first and most likely second plasmon peak
        inelastic       = mean.isig[self.elastic_threshold:]

        axis_inelastic  = inelastic.axes_manager.signal_axes[0]
        amp_pp          = np.max(inelastic, axis=axis_inelastic).data[0]

        param_inelastic = inelastic.find_peaks1D_ohaver(amp_thresh = 0.1*amp_pp,
                                                        maxpeakn   = 2
                                                       )

        if (len(param_inelastic) >= 1):
            fpp_pos     = param_inelastic[0][0][0]
            fpp_fwhm    = param_inelastic[0][0][2]
            
            if (self.function_set == 'VolumePlasmonDrude'):
                # currently intensity set to amplitude as integral of the drude model
                # is not corresponding to the area - might be not working correctly
                # improvement would be needed
                fpp_int = amp_pp
                
            else:
                # estimation by gaussian integration from -fwhm to fwhm is erf(log(4)^(1/2))
                # as all other functions use area under function 
                FPP_signal  = inelastic.isig[fpp_pos - fpp_fwhm : fpp_pos + fpp_fwhm] 
                fpp_axis    = FPP_signal.axes_manager.signal_axes[0]
                fpp_int     = (1 / scipy.special.erf( np.sqrt( np.log(4) ) ) * 
                               FPP_signal.integrate1D(fpp_axis).data[0]
                              )

        else:
            print('Could not estimate initial parameters. Results could diverge!' +
                  'It is highly recommended to adjust the parameters manually.'
                 )
            fpp_pos     = 15. # in eV
            fpp_fwhm    = 2.  # in eV
            
            if (self.function_set == 'VolumePlasmonDrude'):
                # currently intensity set to amplitude as integral of the drude model
                # is not corresponding to the area - might be not working correctly
                # improvement would be needed
                fpp_int = amp_pp
                
            else:
                # estimation by gaussian integration from -fwhm to fwhm is erf(log(4)^(1/2))
                # as all other functions use area under function 
                FPP_signal  = inelastic.isig[fpp_pos - fpp_fwhm : fpp_pos + fpp_fwhm] 
                fpp_axis    = FPP_signal.axes_manager.signal_axes[0]
                fpp_int     = (1 / scipy.special.erf( np.sqrt( np.log(4) ) ) * 
                               FPP_signal.integrate1D(fpp_axis).data[0]
                          )

        if (len(param_inelastic) == 2):
            spp_pos     = param_inelastic[0][1][0]
            spp_fwhm    = param_inelastic[0][1][2]
            
            if (self.function_set == 'VolumePlasmonDrude'):
                # currently intensity set to amplitude as integral of the drude model
                # is not corresponding to the area - might be not working correctly
                # improvement would be needed
                spp_int = amp_pp * 0.3
                
            else:
                # estimation by gaussian integration from -fwhm to fwhm is erf(log(4)^(1/2))
                # as all other functions use area under function 
                SPP_signal  = inelastic.isig[spp_pos - spp_fwhm : spp_pos + spp_fwhm] 
                spp_axis    = SPP_signal.axes_manager.signal_axes[0]
                spp_int     = (1 / scipy.special.erf( np.sqrt( np.log(4) ) ) * 
                               SPP_signal.integrate1D(spp_axis).data[0]
                              )

        else:
            spp_pos     = 2 * fpp_pos
            spp_fwhm    = 2 * fpp_fwhm
            
            if (self.function_set == 'VolumePlasmonDrude'):
                # currently intensity set to amplitude as integral of the drude model
                # is not corresponding to the area - might be not working correctly
                # improvement would be needed
                spp_int = amp_pp * 0.3
                
            else:
                #  estimation by gaussian integration from -fwhm to fwhm is erf(log(4)^(1/2))
                SPP_signal  = inelastic.isig[spp_pos - spp_fwhm : spp_pos + spp_fwhm] 
                spp_axis    = SPP_signal.axes_manager.signal_axes[0]
                spp_int     = (1 / scipy.special.erf( np.sqrt( np.log(4) ) ) * 
                               SPP_signal.integrate1D(spp_axis).data[0]
                              )

        if (fpp_int > zlp_int or spp_int > zlp_int):
            print('Attention: The intensity of one of the Plasmon peaks is\n' + 
                  '           higher than the Zero-loss intensity!\n' + 
                  '           The model should be monitored carefully.\n' +
                  '           It is recommended to gather a new spectrum of\n' +
                  '           a sample with smaller thickness to minimize\n' +
                  '           the inelastically scattered part of the spectrum!'
                 )
        
        return zlp_pos, zlp_fwhm, zlp_int, fpp_pos, fpp_fwhm, fpp_int, spp_pos, spp_fwhm, spp_int
        
        
    def init_model(self, 
                   mean = False
                  ):
        """
        Comments missing
        """
        offset = self.File.axes_manager['Energy loss'].offset
        scale  = self.File.axes_manager['Energy loss'].scale
        size   = self.File.axes_manager['Energy loss'].size
        
        e_max  = float(int(scale * size + offset))
        
        if (mean == True):
            model               = self.File.mean().create_model(
                auto_background = False
            )
        
        elif (mean == False):
            model               = self.File.create_model(
                auto_background = False
            )

        func1, func2, func3 = self.init_func()
        params              = self.init_model_params()
        
        model.set_signal_range(-self.elastic_threshold, 
                               e_max
                              )
        model.extend([func1, func2, func3])

        self.set_model_params(model, 
                              params = params, 
                              func   = 'Zero_Loss_Peak'
                             )
        self.set_model_params(model, 
                              params = params, 
                              func   = 'First_Plasmon_Peak'
                             )
        self.set_model_params(model, 
                              params = params, 
                              func   = 'Second_Plasmon_Peak'
                             )
        
        self.set_second_plasmonenergy(model)
                
        return model, params
    
    
    def set_second_plasmonenergy(self, model):
        """
        Comments missing
        """
        if (self.function_set != 'VolumePlasmonDrude'):
            model.components.Second_Plasmon_Peak.centre.twin_function_expr         = '2*x'
            model.components.Second_Plasmon_Peak.centre.inverse_twin_function_expr = 'x/2'
            model.components.Second_Plasmon_Peak.centre.twin                       = model.components.First_Plasmon_Peak.centre

        else:
            model.components.Second_Plasmon_Peak.plasmon_energy.twin_function_expr          = '2*x'
            model.components.Second_Plasmon_Peak.plasmon_energy.inverse_twin_function_expr  = 'x/2'
            model.components.Second_Plasmon_Peak.plasmon_energy.twin                        = model.components.First_Plasmon_Peak.plasmon_energy
    
    
    def fit_zlp_only(self, 
                     model, 
                     fitter, 
                     method,
                     bounded
                    ):
        """
        Comments missing
        """
        mean, params = self.init_model(mean = True)
        
        self.set_model_params(model, 
                              params = params, 
                              func   = 'Zero_Loss_Peak'
                             )
        
        self.set_bounds(mean, 
                        bounded
                       )
        
        mean.components.Zero_Loss_Peak.active      = True
        mean.components.First_Plasmon_Peak.active  = False
        mean.components.Second_Plasmon_Peak.active = False

        zlp_pos             = params[0]
        zlp_fwhm            = params[1]
        
        mean.set_signal_range(zlp_pos - zlp_fwhm, 
                              zlp_pos + zlp_fwhm
                             )
        
        mean.fit(fitter  = fitter, 
                 method  = method,
                 bounded = bounded
                )
        
        self.set_model_params(model, 
                              mean = mean, 
                              func = 'Zero_Loss_Peak'
                             )
        
        
    def fit_pp_only(self,
                    upper_bound,
                    model, 
                    fitter, 
                    method,
                    bounded
                   ):
        """
        Comments missing
        """
        mean, params = self.init_model(mean = True)
        
        self.set_model_params(model, 
                              params = params, 
                              func   = 'First_Plasmon_Peak'
                             )
        
        self.set_model_params(model, 
                              params = params, 
                              func   = 'Second_Plasmon_Peak'
                             )
        
        self.set_bounds(mean, 
                        bounded
                       )
        
        mean.components.Zero_Loss_Peak.active      = False
        mean.components.First_Plasmon_Peak.active  = True
        mean.components.Second_Plasmon_Peak.active = True

        self.set_second_plasmonenergy(mean)
        
        # used for the signal range to exclude additional influence
        fpp_pos  = params[3]
        fpp_fwhm = params[4]
        spp_pos  = params[6]
        spp_fwhm = params[7]
        
        mean.set_signal_range(fpp_pos - fpp_fwhm, 
                              spp_pos + spp_fwhm
                             )

        mean.fit(fitter=fitter, 
                 method=method,
                 bounded=bounded
                )
        
        self.set_model_params(model, 
                              mean = mean, 
                              func = 'First_Plasmon_Peak'
                             )
        
        self.set_model_params(model, 
                              mean = mean, 
                              func = 'Second_Plasmon_Peak'
                             )
        
        return self.get_plasmonenergy(mean)
        
    
    def fit_fpp_only(self, 
                     upper_bound,
                     model, 
                     fitter, 
                     method,
                     bounded
                    ):
        """
        Comments missing
        """
        mean, params = self.init_model(mean = True)
        
        self.set_bounds(mean, 
                        bounded
                       )
        
        mean.components.Zero_Loss_Peak.active      = False
        mean.components.First_Plasmon_Peak.active  = True
        mean.components.Second_Plasmon_Peak.active = False

        mean.set_signal_range(self.elastic_threshold, #see above
                              upper_bound #eV
                             )
        
        mean.fit(fitter=fitter, 
                 method=method,
                 bounded=bounded
                )
        
        plasmonenergy = self.get_plasmonenergy(mean)
        
        mean.set_signal_range(self.elastic_threshold, #see above
                              2.5 * plasmonenergy 
                              #using better estimate of plasmon energy range to increase accuracy 
                             )

        mean.fit(fitter  = fitter, 
                 method  = method,
                 bounded = bounded
                )
        
        self.set_model_params(model, 
                              mean = mean, 
                              func = 'First_Plasmon_Peak'
                             )
        
        return self.get_plasmonenergy(mean)
        
    
    def get_plasmonenergy(self,
                          mean
                         ):
        """
        Comments missing
        """
        if (self.function_set == 'VolumePlasmonDrude'):
            plasmonenergy = mean.components.First_Plasmon_Peak.plasmon_energy.value
        
        else:
            plasmonenergy = mean.components.First_Plasmon_Peak.centre.value
            
        return plasmonenergy
    
    
    def set_bounds(self, 
                   model, 
                   bounded
                  ):
        """
        Comments missing
        """
        if (self.function_set == 'Voigt'):
            
            if (bounded):
                model.components.Zero_Loss_Peak.area.bmin   = (self.elastic_intensity 
                                                               - self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.area.bmax   = (self.elastic_intensity 
                                                               + self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.centre.bmin = -self.elastic_threshold
                model.components.Zero_Loss_Peak.centre.bmax = self.elastic_threshold
                model.components.Zero_Loss_Peak.FWHM.bmin   = 0.
                model.components.Zero_Loss_Peak.FWHM.bmax   = 2 * self.elastic_threshold
                model.components.Zero_Loss_Peak.gamma.bmin  = 0.
                model.components.Zero_Loss_Peak.gamma.bmax  = 2 * self.elastic_threshold


                model.components.First_Plasmon_Peak.area.bmin   = 0.
                model.components.First_Plasmon_Peak.area.bmax   = (self.elastic_intensity 
                                                                   + self.elastic_intensity / 2
                                                                  )
                model.components.First_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.First_Plasmon_Peak.centre.bmax = 50.
                model.components.First_Plasmon_Peak.FWHM.bmin   = 0.
                model.components.First_Plasmon_Peak.FWHM.bmax   = 50.
                model.components.First_Plasmon_Peak.gamma.bmin  = 0.
                model.components.First_Plasmon_Peak.gamma.bmax  = 50.


                model.components.Second_Plasmon_Peak.area.bmin   = 0.
                model.components.Second_Plasmon_Peak.area.bmax   = (self.elastic_intensity 
                                                                    + self.elastic_intensity / 2
                                                                   )
                model.components.Second_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.Second_Plasmon_Peak.centre.bmax = 100.
                model.components.Second_Plasmon_Peak.FWHM.bmin   = 0.
                model.components.Second_Plasmon_Peak.FWHM.bmax   = 100.
                model.components.Second_Plasmon_Peak.gamma.bmin  = 0.
                model.components.Second_Plasmon_Peak.gamma.bmax  = 100.
                
            
            else:
                model.components.Zero_Loss_Peak.area.ext_force_positive        = True
                model.components.Zero_Loss_Peak.centre.ext_force_positive      = False
                model.components.Zero_Loss_Peak.FWHM.ext_force_positive        = True
                model.components.Zero_Loss_Peak.gamma.ext_force_positive       = True
                model.components.First_Plasmon_Peak.area.ext_force_positive    = True
                model.components.First_Plasmon_Peak.centre.ext_force_positive  = True
                model.components.First_Plasmon_Peak.FWHM.ext_force_positive    = True
                model.components.First_Plasmon_Peak.gamma.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.area.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.centre.ext_force_positive = True
                model.components.Second_Plasmon_Peak.FWHM.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.gamma.ext_force_positive  = True
        
        
        elif (self.function_set == 'Lorentzian'):
            
            if (bounded):
                model.components.Zero_Loss_Peak.area.bmin   = (self.elastic_intensity 
                                                               - self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.area.bmax   = (self.elastic_intensity 
                                                               + self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.centre.bmin = -self.elastic_threshold
                model.components.Zero_Loss_Peak.centre.bmax = self.elastic_threshold
                model.components.Zero_Loss_Peak.FWHM.bmin   = 0.
                model.components.Zero_Loss_Peak.FWHM.bmax   = 2 * self.elastic_threshold
                model.components.Zero_Loss_Peak.gamma.bmin  = 0.
                model.components.Zero_Loss_Peak.gamma.bmax  = 2 * self.elastic_threshold


                model.components.First_Plasmon_Peak.A.bmin      = 0.
                model.components.First_Plasmon_Peak.A.bmax      = (self.elastic_intensity 
                                                                   + self.elastic_intensity / 2
                                                                  )
                model.components.First_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.First_Plasmon_Peak.centre.bmax = 50.
                model.components.First_Plasmon_Peak.gamma.bmin  = 0.
                model.components.First_Plasmon_Peak.gamma.bmax  = 50.


                model.components.Second_Plasmon_Peak.A.bmin      = 0.
                model.components.Second_Plasmon_Peak.A.bmax      = (self.elastic_intensity 
                                                                    + self.elastic_intensity / 2
                                                                   )
                model.components.Second_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.Second_Plasmon_Peak.centre.bmax = 100.
                model.components.Second_Plasmon_Peak.gamma.bmin  = 0.
                model.components.Second_Plasmon_Peak.gamma.bmax  = 100.

            
            else:
                model.components.Zero_Loss_Peak.area.ext_force_positive        = True
                model.components.Zero_Loss_Peak.centre.ext_force_positive      = False
                model.components.Zero_Loss_Peak.FWHM.ext_force_positive        = True
                model.components.Zero_Loss_Peak.gamma.ext_force_positive       = True
                model.components.First_Plasmon_Peak.A.ext_force_positive       = True
                model.components.First_Plasmon_Peak.centre.ext_force_positive  = True
                model.components.First_Plasmon_Peak.gamma.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.A.ext_force_positive      = True
                model.components.Second_Plasmon_Peak.centre.ext_force_positive = True
                model.components.Second_Plasmon_Peak.gamma.ext_force_positive  = True
                
                
        elif (self.function_set == 'Gaussian'):
            
            if (bounded):
                model.components.Zero_Loss_Peak.area.bmin   = (self.elastic_intensity 
                                                               - self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.area.bmax   = (self.elastic_intensity 
                                                               + self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.centre.bmin = -self.elastic_threshold
                model.components.Zero_Loss_Peak.centre.bmax = self.elastic_threshold
                model.components.Zero_Loss_Peak.FWHM.bmin   = 0.
                model.components.Zero_Loss_Peak.FWHM.bmax   = 2 * self.elastic_threshold
                model.components.Zero_Loss_Peak.gamma.bmin  = 0.
                model.components.Zero_Loss_Peak.gamma.bmax  = 2 * self.elastic_threshold


                model.components.First_Plasmon_Peak.A.bmin      = 0.
                model.components.First_Plasmon_Peak.A.bmax      = (self.elastic_intensity 
                                                                   + self.elastic_intensity / 2
                                                                  )
                model.components.First_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.First_Plasmon_Peak.centre.bmax = 50.
                model.components.First_Plasmon_Peak.sigma.bmin  = 0.
                model.components.First_Plasmon_Peak.sigma.bmax  = 50.


                model.components.Second_Plasmon_Peak.A.bmin      = 0.
                model.components.Second_Plasmon_Peak.A.bmax      = (self.elastic_intensity 
                                                                    + self.elastic_intensity / 2
                                                                   )
                model.components.Second_Plasmon_Peak.centre.bmin = self.elastic_threshold
                model.components.Second_Plasmon_Peak.centre.bmax = 100.
                model.components.Second_Plasmon_Peak.sigma.bmin  = 0.
                model.components.Second_Plasmon_Peak.sigma.bmax  = 100.
            

            else:
                model.components.Zero_Loss_Peak.area.ext_force_positive        = True
                model.components.Zero_Loss_Peak.centre.ext_force_positive      = False
                model.components.Zero_Loss_Peak.FWHM.ext_force_positive        = True
                model.components.Zero_Loss_Peak.gamma.ext_force_positive       = True
                model.components.First_Plasmon_Peak.A.ext_force_positive       = True
                model.components.First_Plasmon_Peak.centre.ext_force_positive  = True
                model.components.First_Plasmon_Peak.sigma.ext_force_positive   = True
                model.components.Second_Plasmon_Peak.A.ext_force_positive      = True
                model.components.Second_Plasmon_Peak.centre.ext_force_positive = True
                model.components.Second_Plasmon_Peak.sigma.ext_force_positive  = True
                
            
        elif (self.function_set == 'VolumePlasmonDrude'):
            
            if (bounded):
                model.components.Zero_Loss_Peak.area.bmin   = (self.elastic_intensity 
                                                               - self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.area.bmax   = (self.elastic_intensity 
                                                               + self.elastic_intensity / 2
                                                              )
                model.components.Zero_Loss_Peak.centre.bmin = -self.elastic_threshold
                model.components.Zero_Loss_Peak.centre.bmax = self.elastic_threshold
                model.components.Zero_Loss_Peak.FWHM.bmin   = 0.
                model.components.Zero_Loss_Peak.FWHM.bmax   = 2 * self.elastic_threshold
                model.components.Zero_Loss_Peak.gamma.bmin  = 0.
                model.components.Zero_Loss_Peak.gamma.bmax  = 2 * self.elastic_threshold


                model.components.First_Plasmon_Peak.intensity.bmin      = 0.
                model.components.First_Plasmon_Peak.intensity.bmax      = (self.elastic_intensity 
                                                                           + self.elastic_intensity / 2
                                                                          )
                model.components.First_Plasmon_Peak.plasmon_energy.bmin = self.elastic_threshold
                model.components.First_Plasmon_Peak.plasmon_energy.bmax = 50.
                model.components.First_Plasmon_Peak.fwhm.bmin  = 0.
                model.components.First_Plasmon_Peak.fwhm.bmax  = 50.


                model.components.Second_Plasmon_Peak.intensity.bmin      = 0.
                model.components.Second_Plasmon_Peak.intensity.bmax      = (self.elastic_intensity 
                                                                            + self.elastic_intensity / 2
                                                                           )
                model.components.Second_Plasmon_Peak.plasmon_energy.bmin = self.elastic_threshold
                model.components.Second_Plasmon_Peak.plasmon_energy.bmax = 100.
                model.components.Second_Plasmon_Peak.fwhm.bmin  = 0.
                model.components.Second_Plasmon_Peak.fwhm.bmax  = 100.


            else:
                model.components.Zero_Loss_Peak.area.ext_force_positive                = True
                model.components.Zero_Loss_Peak.centre.ext_force_positive              = False
                model.components.Zero_Loss_Peak.FWHM.ext_force_positive                = True
                model.components.Zero_Loss_Peak.gamma.ext_force_positive               = True
                model.components.First_Plasmon_Peak.intensity.ext_force_positive       = True
                model.components.First_Plasmon_Peak.plasmon_energy.ext_force_positive  = True
                model.components.First_Plasmon_Peak.fwhm.ext_force_positive            = True
                model.components.Second_Plasmon_Peak.intensity.ext_force_positive      = True
                model.components.Second_Plasmon_Peak.plasmon_energy.ext_force_positive = True
                model.components.Second_Plasmon_Peak.fwhm.ext_force_positive           = True
                
    
    def set_by_params(self,
                      model,
                      params,
                      func
                     ):
        """
        Comments missing
        """
        func_dict = {'Zero_Loss_Peak'      : params[0:3],
                     'First_Plasmon_Peak'  : params[3:6],
                     'Second_Plasmon_Peak' : params[6:9]
                    }

        if (self.function_set == 'Voigt' or func == 'Zero_Loss_Peak'):
            model.set_parameters_value('area',
                                       func_dict[func][2],
                                       component_list = [func]
                                      )

            model.set_parameters_value('gamma',
                                       func_dict[func][1] / 2,
                                       component_list = [func]
                                      )

            model.set_parameters_value('FWHM',
                                       func_dict[func][1] / 2,
                                       component_list = [func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func][0],
                                           component_list = [func]
                                          )

        if (self.function_set == 'Lorentzian' and func != 'Zero_Loss_Peak'):
            model.set_parameters_value('A',
                                       func_dict[func][2],
                                       component_list = [func]
                                      )

            model.set_parameters_value('gamma',
                                       func_dict[func][1] / 2,
                                       component_list = [func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func][0],
                                           component_list = [func]
                                          )

        if (self.function_set == 'Gaussian' and func != 'Zero_Loss_Peak'):
            model.set_parameters_value('A',
                                       func_dict[func][2],
                                       component_list = [func]
                                      )

            model.set_parameters_value('sigma',
                                       func_dict[func][1] / (2 * np.sqrt( np.log(2) )),
                                       component_list = [func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func][0],
                                           component_list = [func]
                                          )

        if (self.function_set == 'VolumePlasmonDrude' and func != 'Zero_Loss_Peak'):
            print(func_dict[func][2])
            model.set_parameters_value('intensity',
                                       func_dict[func][2],
                                       component_list = [func]
                                      )

            model.set_parameters_value('fwhm',
                                       func_dict[func][1],
                                       component_list = [func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('plasmon_energy',
                                           func_dict[func][0],
                                           component_list = [func]
                                          )
    
    
    def set_by_mean(self,
                    model,
                    mean,
                    func
                   ):
        """
        Comments missing
        """
        func_dict = {'Zero_Loss_Peak'      : mean.components.Zero_Loss_Peak,
                     'First_Plasmon_Peak'  : mean.components.First_Plasmon_Peak,
                     'Second_Plasmon_Peak' : mean.components.Second_Plasmon_Peak
                    }
        
        if (self.function_set == 'Voigt' or func == 'Zero_Loss_Peak'):
            model.set_parameters_value('area',
                                       func_dict[func].area.value,
                                       component_list = [func]
                                      )

            model.set_parameters_value('gamma',
                                       func_dict[func].gamma.value,
                                       component_list = [func]
                                      )

            model.set_parameters_value('FWHM',
                                       func_dict[func].FWHM.value,
                                       component_list = [func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func].centre.value,
                                           component_list = [func]
                                          )

        if (self.function_set == 'Lorentzian' and func != 'Zero_Loss_Peak'):
            model.set_parameters_value('A',
                                       func_dict[func].A.value,
                                       component_list = [func]
                                      )

            model.set_parameters_value('gamma',
                                       func_dict[func].gamma.value,
                                       component_list = [func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func].centre.value,
                                           component_list = [func]
                                          )

        if (self.function_set == 'Gaussian' and func != 'Zero_Loss_Peak'):
            model.set_parameters_value('A',
                                       func_dict[func].A.value,
                                       component_list = [func]
                                      )

            model.set_parameters_value('sigma',
                                       func_dict[func].sigma.value,
                                       component_list = [func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('centre',
                                           func_dict[func].centre.value,
                                           component_list = [func]
                                          )

        if (self.function_set == 'VolumePlasmonDrude' and func != 'Zero_Loss_Peak'):
            model.set_parameters_value('intensity',
                                       func_dict[func].intensity.value,
                                       component_list = [func]
                                      )

            model.set_parameters_value('fwhm',
                                       func_dict[func].fwhm.value,
                                       component_list = [func]
                                      )

            if (func != 'Second_Plasmon_Peak'):
                model.set_parameters_value('plasmon_energy',
                                           func_dict[func].plasmon_energy.value,
                                           component_list = [func]
                                          )
    
            
    def set_model_params(self, 
                         model, 
                         mean   = None,
                         params = None,
                         func='Zero_Loss_Peak'
                        ):
        """
        Comments missing
        """
        if (params != None):
            self.set_by_params(model, params, func)
            
        elif (mean != None):
            self.set_by_mean(model, mean, func)
    
    
    def fit_eels(self, 
                 fitter, 
                 method
                ):
        """
        Comments missing
        """
        model, params = self.init_model()
        
        if (method == 'ls'):
            bounded = True
        
        else:
            bounded = False
            
        self.set_bounds(model, 
                        bounded
                       )
        
        offset = self.File.axes_manager['Energy loss'].offset
        scale  = self.File.axes_manager['Energy loss'].scale
        size   = self.File.axes_manager['Energy loss'].size
        
        e_max  = float(int(scale * size + offset))
        
        upper_bound = e_max
        
        if (self.attr_deconv == False):
            self.set_second_plasmonenergy(model)
            
            self.fit_zlp_only(model, 
                              fitter, 
                              method,
                              bounded
                             )
            
            plasmonenergy = self.fit_pp_only(upper_bound,
                                             model, 
                                             fitter, 
                                             method,
                                             bounded
                                            )

            model.components.Zero_Loss_Peak.active      = True
            model.components.First_Plasmon_Peak.active  = True
            model.components.Second_Plasmon_Peak.active = True
            
            model.set_signal_range(-self.elastic_threshold, 
                                   2.5 * plasmonenergy
                                  )
            
            print('Correcting poissonian noise for the gain factor of\n' +
                  'the EELS - detector by generating statistics on the\n:' +
                  'reduced Chi-squared.'
                 )
            
            mean_rchisq = self.poisson_noise_gain_correction(model,
                                                             fitter,
                                                             method,
                                                             bounded
                                                            )
            
            model.multifit(fitter  = fitter, 
                           method  = method,
                           bounded = bounded
                          )
        
        elif (self.attr_deconv == True):
            model.components.Zero_Loss_Peak.active      = False
            model.components.First_Plasmon_Peak.active  = True
            model.components.Second_Plasmon_Peak.active = False
            
            plasmonenergy = self.fit_fpp_only(upper_bound,
                                              model, 
                                              fitter, 
                                              method
                                             )
            
            model.set_signal_range(self.elastic_threshold, 
                                   1.5 * plasmonenergy
                                  )
            
            print('Correcting poissonian noise for the gain factor of\n' +
                  'the EELS - detector by generating statistics on the\n:' +
                  'reduced Chi-squared.'
                 )
            
            mean_rchisq = self.poisson_noise_gain_correction(model,
                                                             fitter,
                                                             method,
                                                             bounded
                                                            )
            
            model.multifit(fitter  = fitter, 
                           method  = method,
                           bounded = bounded
                          )
        
        self.Chisq       = model.chisq
        self.red_Chisq   = model.red_chisq
        self.rchisq_mean = np.mean(self.red_Chisq.data[np.invert(np.isnan(self.red_Chisq.data))])
        self.rchisq_std  = np.std(self.red_Chisq.data[np.invert(np.isnan(self.red_Chisq.data))])

        print('Adj. fit goodness (reduced Chi squared): ', self.rchisq_mean)
        print('Standard deviation of adj. fit goodness: ', self.rchisq_std)
        
        return model
    
    
    def fit_eels_SAMF(self, 
                      fitter, 
                      method, 
                      multithreading, 
                      workers
                     ):
        """
        Comments missing
        """
        print('Reinitialising poissonian noise estimation...')
        self.File.estimate_poissonian_noise_variance()
        
        model, params = self.init_model()
        
        if (method == 'ls'):
            bounded = True
        
        else:
            bounded = False
        
        self.set_bounds(model, bounded)
        
        offset = self.File.axes_manager['Energy loss'].offset
        scale  = self.File.axes_manager['Energy loss'].scale
        size   = self.File.axes_manager['Energy loss'].size
        
        e_max  = float(int(scale * size + offset))
        
        upper_bound = e_max
        
        if (self.attr_deconv == False):
            self.set_second_plasmonenergy(model)
            
            self.fit_zlp_only(model, 
                              fitter, 
                              method,
                              bounded
                             )
            
            self.fit_pp_only(upper_bound,
                             model, 
                             fitter, 
                             method,
                             bounded
                            )
            
            if (self.function_set == 'VolumePlasmonDrude'):
                plasmonenergy = np.mean(
                    model.components.First_Plasmon_Peak.plasmon_energy.as_signal(field='values').data
                )
                
            
            else:
                plasmonenergy = np.mean(
                    model.components.First_Plasmon_Peak.centre.as_signal(field='values').data
                )
            
            model.components.Zero_Loss_Peak.active      = True
            model.components.First_Plasmon_Peak.active  = True
            model.components.Second_Plasmon_Peak.active = True
            
            model.set_signal_range(-self.elastic_threshold, 
                                   2.5 * plasmonenergy
                                  )
            
            gui = self.yes_or_no('Do you want to adjust the starting parameters?')
            if (gui == True):
                self.model_gui(model)
            
            print('Correcting poissonian noise for the gain factor of\n' +
                  'the EELS - detector by generating statistics on the\n:' +
                  'reduced Chi-squared.'
                 )
            
            mean_rchisq = self.poisson_noise_gain_correction(model,
                                                             fitter,
                                                             method,
                                                             bounded
                                                            )
            
            print('Generating_seeds for SAMFire...')
            
            self.generate_seeds(model,
                                fitter,
                                method,
                                bounded
                               )
            
            samf = model.create_samfire(workers=workers, 
                                        ipyparallel=multithreading, 
                                        setup=True
                                       )

            samf.metadata.goodness_test.tolerance = mean_rchisq * 1.5
            samf.remove(1)
            samf.refresh_database()

            samf.start(fitter=fitter, 
                       method=method,
                       bounded=bounded
                      )

            plt.close()
            
        
        elif (self.attr_deconv == True):
            model.components.Zero_Loss_Peak.active      = False
            model.components.First_Plasmon_Peak.active  = True
            model.components.Second_Plasmon_Peak.active = False
            
            plasmonenergy = self.fit_fpp_only(upper_bound,
                                              model, 
                                              fitter,
                                              method,
                                              bounded
                                             )
            
            model.set_signal_range(self.elastic_threshold, 
                                   1.5 * plasmonenergy
                                  )
            
            print('Correcting poissonian noise for the gain factor of\n' +
                  'the EELS - detector by generating statistics on the\n' +
                  'reduced Chi-squared.'
                 )
            
            mean_rchisq = self.poisson_noise_gain_correction(model,
                                                             fitter,
                                                             method,
                                                             bounded
                                                            )
            
            print('Generating_seeds for SAMFire...')
            
            self.generate_seeds(model,
                                fitter,
                                method,
                                bounded
                               )
            
            samf = model.create_samfire(workers=workers, 
                                        ipyparallel=multithreading, 
                                        setup=True
                                       )

            samf.metadata.goodness_test.tolerance = mean_rchisq * 1.5
            samf.remove(1)
            samf.refresh_database()

            samf.start(fitter=fitter, 
                       method=method,
                       bounded=bounded
                      )
        
        try:
            self.Chisq       = model.chisq
            self.red_Chisq   = model.red_chisq
            self.rchisq_mean = np.mean(self.red_Chisq.data[np.invert(np.isnan(self.red_Chisq.data))])
            self.rchisq_std  = np.std(self.red_Chisq.data[np.invert(np.isnan(self.red_Chisq.data))])

            print('Adj. fit goodness (reduced Chi squared): ', self.rchisq_mean)
            print('Standard deviation of adj. fit goodness: ', self.rchisq_std)
        except:
            print('did not work')
        return model
    
    
    def generate_seeds(self,
                       model,
                       fitter,
                       method,
                       bounded
                      ):
        """
        Calling serial fit routine for a #number of pixels to gather statistics
        on the reduced chi squared.
        (#number = steps|steps^2, dependence of navigation dimension) 
        
        The pixel selection is homogenously distributed over the navigation space,
        to estimate local deviations of the mean parameter solutions.
        This way, SAMFire will have optimally close solutions for the parameter
        space to estimate neighbouring pixels and propagate fast.
        
        It is a requirement for SAMFire.
        For multifit routine: functionality is only needed for gain correction!
        """
        steps = 10
        
        if (model.axes_manager.navigation_dimension == 1):
            
            for i in range(steps):
                step_x = int(self.File.axes_manager.navigation_shape[0]/steps)

                model.axes_manager.indices = (int(step_x/2+i*step_x),)

                model.fit(fitter=fitter, 
                          method=method,
                          bounded=bounded
                         )


        elif (model.axes_manager.navigation_dimension == 2):

            for i in range(steps):
                for j in range(steps):
                    step_x = int(self.File.axes_manager.navigation_shape[0]/steps)
                    step_y = int(self.File.axes_manager.navigation_shape[1]/steps)

                    model.axes_manager.indices = (int(step_x/2 + i*step_x), 
                                                  int(step_y/2 + j*step_y))

                    model.fit(fitter=fitter, 
                              method=method,
                              bounded=bounded
                             )
                        
        
    def poisson_noise_gain_correction(self,
                                      model,
                                      fitter,
                                      method,
                                      bounded
                                     ):
        """
        Adjusting the poissonian noise by considering the influence of the 
        gain factor, as it is a multiplier attached to the poissonian noise.
        
        This will use the reduced chi squared to estimate the gain factor by
        assuming a optimal model, setting the mean chi squared to 1.
        
        Therefore all information on the fit goodness of the model is adjusted.
        """
        self.generate_seeds(model,
                            fitter,
                            method,
                            bounded
                           )

        red_chisq = model.red_chisq.data
        mean_rchisq = np.mean(red_chisq[np.invert(np.isnan(red_chisq))])
        
        print('Including gain factor of detector in poissonian noise. Rescaling...')
        self.File.estimate_poissonian_noise_variance(gain_factor=mean_rchisq)
        return mean_rchisq
    
    
    def save_models_to_file(self, 
                            filename
                           ):
        """
        Comments missing
        """
        self.File.save(filename)

    
    def load_model(self):
        """
        Comments missing
        """
        print('Modelinformation of loaded file: \n' + str(self.File.models))
        available = np.full((12), True, dtype=bool)
        print('Available models: \n')
        
        #Check existing models
        for i in range(len(available)):
            try:
                self.model_name = self.models_dict[str(i)]
                self.restore_model()
            except:
                available[i] = False
            
            if (available[i] == True):
                print('For ' + self.models_dict[str(i)] + 
                      ' type: "' + str(i) + 
                      '" or the full model name.\n'
                     )
        
        if (available[available].size == 0):
            print('No model is available. Exiting.')
            return None
        
        print('\nIf you want to exit the model loading process,' + 
              'please type: ("exit"/"cancel")'
             )
        mkey = input("Which model should be loaded? ")

        if (mkey == 'exit' or mkey == 'cancel'):
            print('Loading process is interrupted by user input.')
            return None
        
        elif (mkey in self.models_dict and available[int(mkey)] == True):
            print('Loading parameter maps for: ' + 
                  str(self.models_dict[mkey])
                 )
            self.model_name = self.models_dict[mkey]

            self.restore_model()
            self.function_set = self.split(self.models_dict[mkey], ('_'))[0]
            self.Chisq        = self.Fit_Model.chisq
            self.red_Chisq    = self.Fit_Model.red_chisq

        else:
            try:
                print('Loading parameter maps for: ' + str(mkey))
                self.model_name   = mkey

                self.restore_model()
                self.function_set = self.split(self.models_dict[mname], ('_'))[0]
                self.Chisq        = self.Fit_Model.chisq
                self.red_Chisq    = self.Fit_Model.red_chisq
            
            except:
                print('Your input did not match with any existing model of the loaded file, please try again.')
                self.load_model()
        
        method = self.split(self.models_dict[mkey], ('_'))[-1]
        self.generate_param_maps(method)

    
    def restore_model(self):
        """
        Comments missing
        """
        self.Fit_Model  = self.File.models.restore(self.model_name)
            
    
    def voigt_fwhm_gauss_propagation(self,
                                     func
                                    ):
        # fwhm - gaussian error propagation for voigt
        func_dict = {'Zero_Loss_Peak'      : self.Fit_Model.components.Zero_Loss_Peak,
                     'First_Plasmon_Peak'  : self.Fit_Model.components.First_Plasmon_Peak,
                     'Second_Plasmon_Peak' : self.Fit_Model.components.Second_Plasmon_Peak
                    }

        gamma_zlp = func_dict[func].gamma.as_signal(
            field = 'values'
        )

        L_fwhm    = np.add(gamma_zlp, 
                           gamma_zlp
                          )

        G_fwhm    = func_dict[func].FWHM.as_signal(
            field = 'values'
        )

        gamma_zlp = func_dict[func].gamma.as_signal(
            field = 'std'
        )

        L_fwhm_std = np.add(gamma_zlp, 
                            gamma_zlp
                           )

        G_fwhm_std = func_dict[func].FWHM.as_signal(
            field = 'std'
        )

        dfwhm_dl = (L_fwhm * 0.2166 / ( L_fwhm**2 * 0.2166 + 
                                       G_fwhm**2 )**(1/2) + 
                    0.5346
                   )

        dfwhm_dg = (G_fwhm / ( L_fwhm ** 2 * 0.2166 + 
                              G_fwhm ** 2 )**(1/2)          
                   )

        error = np.sqrt((G_fwhm_std * dfwhm_dg) ** 2 + 
                        (L_fwhm_std * dfwhm_dl) ** 2 
                       )

        return error
    
    
    def Ep_q0_gauss_propagation(self):
        emax  = self.FPP_Emax 
        fwhm  = self.FPP_FWHM
        
        Ep_q0 = ( emax ** 2 - (fwhm / 2) ** 2 ) ** 0.5
        
        dEp_demax = emax * 2 / ( emax ** 2 * 4 + 
                                fwhm ** 2 
                               ) ** (1/2)
        dEp_dfwhm = fwhm * (1/2) / ( emax ** 2 * 4 + 
                                  fwhm ** 2 
                                 ) ** (1/2)
        
        emax_std = self.FPP_Emax.metadata.Signal.Noise_properties.variance ** (1/2)
        fwhm_std = self.FPP_FWHM.metadata.Signal.Noise_properties.variance ** (1/2)
        
        Ep_q0_std = ( ( dEp_demax * emax_std ) ** 2 + ( dEp_dfwhm * fwhm_std ) ** 2 ) ** (1/2)
        
        return Ep_q0_std
        
    
    def param_maps_drude(self):
        self.FPP_FWHM   = self.Fit_Model.components.First_Plasmon_Peak.fwhm.as_signal(
            field = 'values'
        )
        
        self.FPP_Emax   = self.Fit_Model.components.First_Plasmon_Peak.plasmon_energy.as_signal(
            field = 'values'
        )
        
        self.FPP_Int    = self.Fit_Model.components.First_Plasmon_Peak.intensity.as_signal(
            field = 'values'
        )

        if (self.attr_deconv == False):
            gamma_zlp = self.Fit_Model.components.Zero_Loss_Peak.gamma.as_signal(
                field = 'values'
            )
            
            ZLP_L_fwhm      = np.add(gamma_zlp, 
                                     gamma_zlp
                                    )
            
            ZLP_G_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.FWHM.as_signal(
                field = 'values'
            )
            
            self.ZLP_FWHM   = (ZLP_L_fwhm*0.5346 + 
                               np.sqrt(ZLP_G_fwhm**2 + ZLP_L_fwhm**2*0.2166)
                              )
            
            self.ZLP_Emax   = self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(
                field = 'values'
            )
            
            self.ZLP_Int    = self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(
                field = 'values'
            )
            

            self.SPP_FWHM   = self.Fit_Model.components.Second_Plasmon_Peak.fwhm.as_signal(
                field = 'values'
            )
            
            self.SPP_Emax   = self.Fit_Model.components.Second_Plasmon_Peak.plasmon_energy.as_signal(
                field = 'values'
            )
            
            self.SPP_Int    = self.Fit_Model.components.Second_Plasmon_Peak.intensity.as_signal(
                field = 'values'
            )

        self.Ep_q0      = ( self.FPP_Emax**2 - (self.FPP_FWHM / 2)**2 )**0.5
        
    
    def std_maps_drude(self):
        self.FPP_FWHM.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.fwhm.as_signal(
                field = 'std'
            ) ** 2
        )
        
        self.FPP_Emax.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.plasmon_energy.as_signal(
                field = 'std'
            ) ** 2
        )
        
        self.FPP_Int.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.intensity.as_signal(
                field = 'std'
            ) ** 2
        )

        if (self.attr_deconv == False):
            error = self.voigt_fwhm_gauss_propagation('Zero_Loss_Peak')
            
            self.ZLP_FWHM.metadata.Signal.set_item(
                "Noise_properties.variance", 
                error ** 2
            )
            
            self.ZLP_Emax.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.ZLP_Int.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(
                    field = 'std'
                ) ** 2
            )

            self.SPP_FWHM.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.fwhm.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.SPP_Emax.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.plasmon_energy.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.SPP_Int.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.intensity.as_signal(
                    field = 'std'
                ) ** 2
            )
        

        Ep_q0_std = self.Ep_q0_gauss_propagation()
        
        self.Ep_q0.metadata.Signal.set_item(
            "Noise_properties.variance", 
            Ep_q0_std ** 2
        )
        
    
    def param_maps_lorentzian(self):
        gamma_fpp = self.Fit_Model.components.First_Plasmon_Peak.gamma.as_signal(
            field = 'values'
        )
        
        self.FPP_FWHM   = np.add(gamma_fpp, gamma_fpp)
        
        self.FPP_Emax   = self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(
            field = 'values'
        )
        
        self.FPP_Int    = self.Fit_Model.components.First_Plasmon_Peak.A.as_signal(
            field = 'values'
        )

        if (self.attr_deconv == False):
            gamma_zlp = self.Fit_Model.components.Zero_Loss_Peak.gamma.as_signal(
                field = 'values'
            )
            
            ZLP_L_fwhm      = np.add(gamma_zlp, 
                                     gamma_zlp
                                    )
            
            ZLP_G_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.FWHM.as_signal(
                field = 'values'
            )
            
            self.ZLP_FWHM   = (ZLP_L_fwhm*0.5346 + 
                               np.sqrt(ZLP_G_fwhm**2 + 
                                       ZLP_L_fwhm**2*0.2166
                                      )
                              )
            
            self.ZLP_Emax   = self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(
                field = 'values'
            )
            
            self.ZLP_Int    = self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(
                field = 'values'
            )
            

            gamma_spp = self.Fit_Model.components.Second_Plasmon_Peak.gamma.as_signal(
                field = 'values'
            )
            
            self.SPP_FWHM   = np.add(gamma_spp, 
                                     gamma_spp
                                    )
            
            self.SPP_Emax   = self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(
                field = 'values'
            )
            
            self.SPP_Int    = self.Fit_Model.components.Second_Plasmon_Peak.A.as_signal(
                field = 'values'
            )
        

        self.Ep_q0      = ( self.FPP_Emax**2 - (self.FPP_FWHM / 2)**2 )**0.5
    
    
    def std_maps_lorentzian(self):
        self.FPP_FWHM.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.gamma.as_signal(
                field = 'std'
            ) ** 2
        )
        
        self.FPP_Emax.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(
                field = 'std'
            ) ** 2
        )
        
        self.FPP_Int.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.A.as_signal(
                field = 'std'
            ) ** 2
        )

        if (self.attr_deconv == False):
            error = self.voigt_fwhm_gauss_propagation('Zero_Loss_Peak')
            
            self.ZLP_FWHM.metadata.Signal.set_item(
                "Noise_properties.variance", 
                error ** 2
            )
            
            self.ZLP_Emax.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.ZLP_Int.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            
            self.SPP_FWHM.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.gamma.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.SPP_Emax.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.SPP_Int.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.A.as_signal(
                    field = 'std'
                ) ** 2
            )
        

        Ep_q0_std = self.Ep_q0_gauss_propagation()
        
        self.Ep_q0.metadata.Signal.set_item(
            "Noise_properties.variance", 
            Ep_q0_std ** 2
        )
        
    
    def param_maps_gaussian(self):
        self.FPP_FWHM   = self.Fit_Model.components.First_Plasmon_Peak.sigma.as_signal(
                field = 'values'
        )*2*np.sqrt(np.log(2))
        
        self.FPP_Emax   = self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(
            field = 'values'
        )
        
        self.FPP_Int    = self.Fit_Model.components.First_Plasmon_Peak.A.as_signal(
            field = 'values'
        )

        if (self.attr_deconv == False):
            gamma_zlp = self.Fit_Model.components.Zero_Loss_Peak.gamma.as_signal(
                field = 'values'
            )
            
            ZLP_L_fwhm      = np.add(gamma_zlp, 
                                     gamma_zlp
                                    )
            
            ZLP_G_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.FWHM.as_signal(
                field = 'values'
            )
            
            self.ZLP_FWHM   = (ZLP_L_fwhm*0.5346 + 
                               np.sqrt(ZLP_G_fwhm**2 + ZLP_L_fwhm**2*0.2166)
                              )
            
            self.ZLP_Emax   = self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(
                field = 'values'
            )
            
            self.ZLP_Int    = self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(
                field = 'values'
            )

            
            self.SPP_FWHM   = self.Fit_Model.components.Second_Plasmon_Peak.sigma.as_signal(
                field = 'values'
            )*2*np.sqrt(np.log(2))
            
            self.SPP_Emax   = self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(
                field = 'values'
            )
            
            self.SPP_Int    = self.Fit_Model.components.Second_Plasmon_Peak.A.as_signal(
                field = 'values'
            )
        

        self.Ep_q0      = ( self.FPP_Emax**2 - (self.FPP_FWHM / 2)**2 )**0.5
        
    
    def std_maps_gaussian(self):
        self.FPP_FWHM.metadata.Signal.set_item(
            "Noise_properties.variance", 
            sigma_fpp_std = self.Fit_Model.components.First_Plasmon_Peak.sigma.as_signal(
                field = 'std'
            ) ** 2
        )
        
        self.FPP_Emax.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(
                field = 'std'
            ) ** 2
        )
        
        self.FPP_Int.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.A.as_signal(
                field = 'std'
            ) ** 2
        )

        if (self.attr_deconv == False):
            error = self.voigt_fwhm_gauss_propagation('Zero_Loss_Peak')
            
            self.ZLP_FWHM.metadata.Signal.set_item(
                "Noise_properties.variance", 
                error ** 2
            )
            
            self.ZLP_Emax.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.ZLP_Int.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(
                    field = 'std'
                ) ** 2
            )

            
            self.SPP_FWHM.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.gamma.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.SPP_Emax.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.SPP_Int.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.A.as_signal(
                    field = 'std'
                ) ** 2
            )

        Ep_q0_std = self.Ep_q0_gauss_propagation()
        
        self.Ep_q0.metadata.Signal.set_item(
            "Noise_properties.variance", 
            Ep_q0_std ** 2
        )
    
    
    def param_maps_voigt(self):
        gamma_fpp = self.Fit_Model.components.First_Plasmon_Peak.gamma.as_signal(
                    field = 'values'
        )
        FPP_L_fwhm      = np.add(gamma_fpp, 
                                 gamma_fpp
                                )
        FPP_G_fwhm      = self.Fit_Model.components.First_Plasmon_Peak.FWHM.as_signal(
            field = 'values'
        )
        self.FPP_FWHM   = (FPP_L_fwhm*0.5346 + 
                           np.sqrt(FPP_G_fwhm**2 + 
                                   FPP_L_fwhm**2*0.2166
                                  )
                          )
        self.FPP_Emax   = self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(
            field = 'values'
        )
        self.FPP_Int    = self.Fit_Model.components.First_Plasmon_Peak.area.as_signal(
            field = 'values'
        )

        if (self.attr_deconv == False):
            gamma_zlp = self.Fit_Model.components.Zero_Loss_Peak.gamma.as_signal(
                field = 'values'
            )
            ZLP_L_fwhm      = np.add(gamma_zlp, 
                                     gamma_zlp
                                    )
            ZLP_G_fwhm      = self.Fit_Model.components.Zero_Loss_Peak.FWHM.as_signal(
                field = 'values'
            )
            self.ZLP_FWHM   = (ZLP_L_fwhm*0.5346 + 
                               np.sqrt(ZLP_G_fwhm**2 + 
                                       ZLP_L_fwhm**2*0.2166
                                      )
                              )
            self.ZLP_Emax   = self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(
                field = 'values'
            )
            self.ZLP_Int    = self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(
                field = 'values'
            )

            gamma_spp = self.Fit_Model.components.Second_Plasmon_Peak.gamma.as_signal(
                field = 'values'
            )
            SPP_L_fwhm      = np.add(gamma_spp, 
                                     gamma_spp
                                    )
            SPP_G_fwhm      = self.Fit_Model.components.Second_Plasmon_Peak.FWHM.as_signal(
                field = 'values'
            )
            self.SPP_FWHM   = (SPP_L_fwhm*0.5346 + 
                               np.sqrt(SPP_G_fwhm**2 + 
                                       SPP_L_fwhm**2*0.2166
                                      )
                              )
            self.SPP_Emax   = self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(
                field = 'values'
            )
            self.SPP_Int    = self.Fit_Model.components.Second_Plasmon_Peak.area.as_signal(
                field = 'values'
            )

        self.Ep_q0      = ( self.FPP_Emax**2 - (self.FPP_FWHM / 2)**2 )**0.5
        
    
    def std_maps_voigt(self):
        error = self.voigt_fwhm_gauss_propagation('First_Plasmon_Peak')
        
        self.FPP_FWHM.metadata.Signal.set_item(
            "Noise_properties.variance", 
            error ** 2
        )
        
        self.FPP_Emax.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.centre.as_signal(
                field = 'std'
            ) ** 2
        )
        
        self.FPP_Int.metadata.Signal.set_item(
            "Noise_properties.variance", 
            self.Fit_Model.components.First_Plasmon_Peak.area.as_signal(
                field = 'std'
            ) ** 2
        )

        if (self.attr_deconv == False):
            error = self.voigt_fwhm_gauss_propagation('Zero_Loss_Peak')
            
            self.ZLP_FWHM.metadata.Signal.set_item(
                "Noise_properties.variance", 
                error ** 2
            )
            
            self.ZLP_Emax.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Zero_Loss_Peak.centre.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.ZLP_Int.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Zero_Loss_Peak.area.as_signal(
                    field = 'std'
                ) ** 2
            )

            
            error = self.voigt_fwhm_gauss_propagation('Second_Plasmon_Peak')
            
            self.ZLP_FWHM.metadata.Signal.set_item(
                "Noise_properties.variance", 
                error ** 2
            )
            
            self.SPP_Emax.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.centre.as_signal(
                    field = 'std'
                ) ** 2
            )
            
            self.SPP_Int.metadata.Signal.set_item(
                "Noise_properties.variance", 
                self.Fit_Model.components.Second_Plasmon_Peak.area.as_signal(
                    field = 'std'
                ) ** 2
            )

        Ep_q0_std = self.Ep_q0_gauss_propagation()
        
        self.Ep_q0.metadata.Signal.set_item(
            "Noise_properties.variance", 
            Ep_q0_std ** 2
        )


    def generate_param_maps(self, 
                            method
                           ):
        """
        Generating parameter maps to estimate the plasmonic peakshift,
        which is calculated by [Egerton] as follows:  
                        
            $E_p (q=0) = \sqrt{ (E_max)^2 + (\frac{\hbar \Gamma}{2}) }$
        
        
         Standard deviation estimation of parameters by fitting is only
         supported by wheighted least square method.
         It uses gaussian error propagation for some functional dependencies.
         E.g.: The uncertainty for $E_p (q=0)$ is calculated by gaussian
                error propagation (the standard deviations of the variables
                                   are propagated)
        """
        if (self.function_set == 'VolumePlasmonDrude'):
            self.param_maps_drude()
            
            if (method == 'ls'):
                self.std_maps_drude()
            
        elif (self.function_set == 'Lorentzian'):
            self.param_maps_lorentzian()
            if (method == 'ls'):
                self.std_maps_lorentzian()
        
        elif (self.function_set == 'Gaussian'):
            self.param_maps_gaussian()
            if (method == 'ls'):
                self.std_maps_gaussian()
            
        elif (self.function_set == 'Voigt'):
            self.param_maps_voigt()
            if (method == 'ls'):
                self.std_maps_voigt()
        
        else:
            print('No valid function set specified. Please look into docstring for further information.')
        
        
        if (self.attr_deconv == False):
            self.param_dict        = { '$E_{max}$ - Plasmon Peak'              : self.FPP_Emax,
                                       '$\Gamma$ - Plasmon Peak'               : self.FPP_FWHM,
                                       'Intensity - Plasmon Peak'              : self.FPP_Int,

                                       '$E_{max}$ - Zero Loss Peak'            : self.ZLP_Emax,
                                       '$\Gamma$ - Zero Loss Peak'             : self.ZLP_FWHM,
                                       'Intensity - Zero Loss Peak'            : self.ZLP_Int,

                                       '$E_{max}$ - Second Order Plasmon Peak' : self.SPP_Emax,
                                       '$\Gamma$ - Second Order Plasmon Peak'  : self.SPP_FWHM,
                                       'Intensity - Second Order Plasmon Peak' : self.SPP_Int,

                                       '$E_p(q=0)$ - Plasmon Energy'           : self.Ep_q0,
                                       '$I_{PP}/I_{ZLP}$ - Intensity Ratio'    : self.FPP_Int / self.ZLP_Int
                                     }
            for title in self.param_dict:
                self.param_dict[title].metadata.General.title = title
            
        else:
            self.param_dict_deconv = { '$E_{max}$ - Plasmon Peak'              : self.FPP_Emax,
                                       '$\Gamma$ - Plasmon Peak'               : self.FPP_FWHM,
                                       'Intensity - Plasmon Peak'              : self.FPP_Int,

                                       '$E_p(q=0)$ - Plasmon Energy'           : self.Ep_q0
                                     }
            for title in self.param_dict_deconv:
                self.param_dict_deconv[title].metadata.General.title = title
    
    
    def plot_file(self, 
                  darkfield = False,
                  slider    = False
                 ):
        """
        Comments missing
        """
        if (self.Darkfield != None and darkfield == True):
            self.File.plot(navigator = self.Darkfield)
            
        elif (slider == True):
            self.File.plot(navigator = 'slider')
        
        else:
            self.File.plot()
        
    
    def plot_histogramm(self):
        """
        Comments missing
        """
        a.File.get_histogram().plot()
            
    
    def plot_model(self,
                   navigator       = True,
                   show_components = False
                  ):
        """
        Comments missing
        """
        self.Fit_Model.plot(navigator       = navigator,
                            plot_components = show_components
                           )
        

    def plot_parameter_maps(self,
                            first_plasmon_only = True,
                            cmap               = 'hot'
                           ):
        """
        Visualize all parameter maps that were previously generated.
        """
        if (self.attr_deconv == False and first_plasmon_only == False):
            for title in self.param_dict:
                mean = self.param_dict[title].mean(axis=(0,1)).data[0]
                std  = self.param_dict[title].std(axis=(0,1)).data[0]

                self.param_dict[title].plot(vmin = mean - 2 * std, 
                                            vmax = mean + 2 * std, 
                                            cmap = cmap,
                                            axes_ticks=True
                                            #, scalebar_color='black'
                                           )
                
        else:
            for title in self.param_dict_deconv:
                mean = self.param_dict_deconv[title].mean(axis=(0,1)).data[0]
                std  = self.param_dict_deconv[title].std(axis=(0,1)).data[0]

                self.param_dict_deconv[title].plot(vmin = mean - 2 * std, 
                                                   vmax = mean + 2 * std,
                                                   cmap = cmap,
                                                   axes_ticks=True
                                                   #, scalebar_color='black'
                                                  )
    
    
    def print_stats(self):
        """
        Printing standard file information.
        """
        print('Statistics of loaded spectrum image:')
        self.File.print_summary_statistics()
        print('Statistics of deconvolved spectrum image:')
        self.File_deconv.print_summary_statistics()
    
    
    def print_param_stats(self):
        """
        Printing standard parameter information
        """
        if (self.attr_deconv == False):
            for title in self.param_dict:
                print(title)
                self.param_dict[title].print_summary_statistics()
            
        else:
            for title in self.param_dict_deconv:
                print(title)
                self.param_dict_deconv[title].print_summary_statistics()
    
    
    def create_dirs(self):
        """
        Creating directories and sub-directories for evaluation storage.
        """
        for directory in self.dir_list:
            if not os.path.exists(os.getcwd() + os.sep + directory):
                os.makedirs(os.getcwd() + os.sep + directory)
    
    
    def save_parameter_maps(self,
                            dpi=300, 
                            fileformat='png'
                           ):
        """
        Saving function to save all generated parameter maps that were previously generated.
        """
        self.create_dirs()
        
        dir_fname = {'$E_{max}$ - Plasmon Peak [eV]'         : 'FPP_Emax',
                     '$\Gamma$ - Plasmon Peak [eV]'          : 'FPP_FWHM',
                     'Intensity - Plasmon Peak'              : 'FPP_Int' ,
            
                     '$E_{max}$ - Zero Loss Peak [eV]'       : 'ZLP_Emax',
                     '$\Gamma$ - Zero Loss Peak [eV]'        : 'ZLP_FWHM',
                     'Intensity - Zero Loss Peak'            : 'ZLP_Int' ,
                     
                     '$E_{max}$ - Second Plasmon Peak [eV]'  : 'SPP_Emax',
                     '$\Gamma$ -  Second Plasmon Peak [eV]'  : 'SPP_FWHM',
                     'Intensity - Second Plasmon Peak'       : 'SPP_Int' ,
                     
                     '$E_p(q=0)$ - Plasmon Energy [eV]'      : 'Ep_q0'
                    }
        
        if (self.attr_deconv == False):
            for title in self.param_dict:
                fname = os.getcwd() + os.sep + self.dir_list[0] + os.sep + dir_fname[title]
                self.param_dict[title].metadata.General.title = title
                self.param_dict[title].plot(scalebar_color='black')
                plt.savefig(fname, dpi=dpi, extension=fileformat)
                plt.close()
                
        else:
            for title in self.param_dict_deconv:
                fname = os.getcwd() + os.sep + self.dir_list[0] + os.sep + dir_fname[title]
                self.param_dict_deconv[title].metadata.General.title = title
                self.param_dict_deconv[title].plot(scalebar_color='black')
                plt.savefig(fname, dpi=dpi, extension=fileformat)
                plt.close()

        fname = os.getcwd() + os.sep + 'Model_Chisq'
        self.Chisq.metadata.General.title = 'Goodness of Fit: $\Chi ^2$'
        self.Chisq.plot(scalebar_color='black')
        plt.savefig(fname, dpi=dpi, extension=fileformat)
        plt.close()
        
        fname = os.getcwd() + os.sep + self.dir_list[0] + os.sep + 'Model_red_Chisq'
        self.red_Chisq.metadata.General.title = 'Goodness of Fit: $\Chi_{\nu} ^2$'
        self.red_Chisq.plot(scalebar_color='black')
        plt.savefig(fname, dpi=dpi, extension=fileformat)
        plt.close()
        
        fname = os.getcwd() + os.sep + self.dir_list[1] + os.sep + 'Foil_thickness'
        self.thickness_map.plot(scalebar_color='black')
        plt.savefig(fname, dpi=dpi, extension=fileformat)
        plt.close()
        
    
    def line_roi(self, 
                 param_map,
                 interactive=False,
                 width=0
                ):
        """
        Comments missing
        """
        if (self.line == None):
            x1 = 0 
            y1 = 0 
            x2 = 15 
            y2 = 15
            
            self.line = hs.roi.Line2DROI(x1 * param_map.axes_manager[0].scale + 
                                         param_map.axes_manager[0].offset,
                                         y1 * param_map.axes_manager[1].scale + 
                                         param_map.axes_manager[1].offset,
                                         x2 * param_map.axes_manager[0].scale + 
                                         param_map.axes_manager[0].offset,
                                         y2 * param_map.axes_manager[1].scale + 
                                         param_map.axes_manager[1].offset,
                                         linewidth = width * param_map.axes_manager[0].scale
                                        )
        
        interactive = self.yes_or_no('Do you want to use the interactive gui?' + '\n' +
                                     'Adjusting position is possible using the interactive gui.'
                                    )
        
        if (interactive == True):
            param_map.plot(scalebar_color='black')
            return self.line.interactive(param_map)
            
        else:
            return self.line(param_map)

            
    def line_std(self, 
                 param_map
                ):
        """
        Comments missing
        """
        if (self.line == None):
            print('No line is defined. No calculation of standard deviation is possible.')
            return
        
        angle = self.line.angle()
        width = self.line.linewidth
        
        line_std = self.line_scan.deepcopy()
        
        for i in range(len(self.line_scan.data)):
            print('next:')
            coord_x = self.line.x1 + i * param_map.axes_manager[0].scale * np.cos(angle)
            coord_y = self.line.y1 + i * param_map.axes_manager[0].scale * np.sin(angle)
            print(coord_x, coord_y)
            x1_std = coord_x + width/2 * np.sin(angle)
            y1_std = coord_y + width/2 * np.cos(angle)
            x2_std = coord_x - width/2 * np.sin(angle)
            y2_std = coord_y - width/2 * np.cos(angle)
            
            tmp_line = hs.roi.Line2DROI(x1_std,
                                        y1_std,
                                        x2_std,
                                        y2_std,
                                        linewidth = self.line_scan.axes_manager[0].scale
                                       )
            print(tmp_line)
            std_coords  = tmp_line(param_map).std(axis=0).data[0]
            print(std_coords)
            line_std.data[i] = std_coords
        
        return line_std
            
    
    def rect_roi(self, 
                 param_map,
                 interactive=False,
                ):
        """
        Comments missing
        """
        if (self.roi == None):
            left   = 0 
            top    = 0 
            right  = 15 
            bottom = 15
            
            self.roi = hs.roi.RectangularROI(param_map.axes_manager.signal_axes[0].offset +
                                             left   * param_map.axes_manager.signal_axes[0].scale,
                                             param_map.axes_manager.signal_axes[1].offset +
                                             top    * param_map.axes_manager.signal_axes[1].scale,
                                             param_map.axes_manager.signal_axes[0].offset +
                                             right  * param_map.axes_manager.signal_axes[0].scale,
                                             param_map.axes_manager.signal_axes[1].offset +
                                             bottom * param_map.axes_manager.signal_axes[1].scale
                                            )
        
        interactive = self.yes_or_no('Do you want to use the interactive gui?' + '\n' +
                                     'Adjusting position is possible using the interactive gui.'
                                    )
        if (interactive == True):
            param_map.plot(scalebar_color='black')
            return self.roi.interactive(param_map)
            
        else:
            return self.roi(param_map)
            
            
    def generate_linescan(self,
                          param_map,
                          interactive = False,
                          roi         = False,
                          width       = 5
                         ):
        """
        Comments missing
        """        
        line_scan = self.line_roi(param_map, 
                                  interactive = interactive, 
                                  width       = width
                                 )
        
        line_std  = self.line_std(param_map)
        line_scan.metadata.Signal.set_item("Noise_properties.variance", 
                                           line_std
                                          )
        line_scan.metadata.General.title = (param_map.metadata.General.title + 
                                            str(' - line scan')
                                           )
        
        self.linescans[param_map.metadata.General.title] = line_scan
        
    
    # Calculate thickness by log ratio of ZLP-Area to Total Area of EELS-data
    def calc_thickness(self, elements, concentrations):
        """
        Estimation of the sample thickness by the Log-Ratio method
                
            LATEX Formula:
                t = \lambda \ln{ \frac{I_t}{I_0} }
                
                with:
                         I_t = total intensity
                    and  I_0 = elastically scattered intensity
        
        For more information see:
            EELS Log-Ratio Technique for Specimen-Thickness
            Measurement in the TEM                    
        
            MALIS, S.C. CHENG, AND R.F. EGERTON
            JOURNAL OF ELECTRON MICROSCOPY TECHNIQUE 8:193-200 11988)
        
        Mean Free Path (\lambda) estimation is automated for a given elemental 
        composition.
        
        IMPORTANT:
            MFP-Automation takes metadata of file as input. Please verify that 
            values for beam energy and collection angle specified in .dm3/.dm4 
            files are correct before using this function.
        
        Currently supported elements: 
        (
         Ag, Al, Au, Be, Ca, Ce, Cu, Dy, Er, Eu, Fe, Gd, Ho, La, Lu, Mg, Nb, 
         Nd, Ni, P, Pd, Pm, Pr, Sm, Sn, Tb, Ti, Y, Yb, Zn, Zr
        )
        """
        t_lambda           = self.File.estimate_thickness(threshold=self.elastic_threshold).T
        mfp                = self.estimate_MFP(elements, concentrations)
        self.thickness_map = t_lambda * mfp
        
        self.thickness_map.metadata.General.title = r'thickness map $t = \lambda \cdot \ln{\frac{I_t}{I_0}}$' 
    
    
    def estimate_MFP(self, elements, concentrations):#, elements, concentration):
        """
        Estimation of the Mean Free Path for a given elemental composition.
        
        IMPORTANT:
            MFP-Automation takes metadata of file as input. Please verify that 
            values for beam energy and collection angle specified in .dm3/.dm4 
            files are correct before using this function.
            Default values (if not specified in metadata):
                E0   = 300 keV
                beta =  10 mrad
        
        Currently supported elements: 
        (
         Ag, Al, Au, Be, Ca, Ce, Cu, Dy, Er, Eu, Fe, Gd, Ho, La, Lu, Mg, Nb, 
         Nd, Ni, P, Pd, Pm, Pr, Sm, Sn, Tb, Ti, Y, Yb, Zn, Zr
        )
        """
        
        element_dict = {'Ag' : mend.Ag,
                        'Al' : mend.Au,
                        'Au' : mend.Au,
                        'Be' : mend.Be,
                        'Ca' : mend.Ca,
                        'Ce' : mend.Ce,
                        'Cu' : mend.Cu,
                        'Dy' : mend.Dy,
                        'Er' : mend.Er,
                        'Eu' : mend.Eu,
                        'Fe' : mend.Fe,
                        'Gd' : mend.Gd,
                        'Ho' : mend.Ho,
                        'La' : mend.La,
                        'Lu' : mend.Lu,
                        'Mg' : mend.Mg,
                        'Nb' : mend.Nb,
                        'Nd' : mend.Nd,
                        'Ni' : mend.Ni,
                        'P'  : mend.P,
                        'Pd' : mend.Pd,
                        'Pm' : mend.Pm,
                        'Pr' : mend.Pr,
                        'Pt' : mend.Pt,
                        'Sm' : mend.Sm,
                        'Sn' : mend.Sn,
                        'Tb' : mend.Tb,
                        'Ti' : mend.Ti,
                        'Tm' : mend.Tm,
                        'Y'  : mend.Y,
                        'Yb' : mend.Yb,
                        'Zn' : mend.Zn,
                        'Zr' : mend.Zr
                       }
        
        const              = 0.3 # Malis EELS paper - konstante r nach eq.(4)
        number_of_elements = len(elements)
        fi_Zi_numerator    = 0
        fi_Zi_denominator  = 0

        for i in range(number_of_elements):
            
            Z                  = element_dict[elements[i]].atomic_number
            fi_Zi_numerator   += concentrations[i] * Z**(1+const)
        
        
        for i in range(number_of_elements):
            
            Z                  = element_dict[elements[i]].atomic_number
            fi_Zi_denominator += concentrations[i] * Z**(const)

        Z_eff = fi_Zi_numerator/fi_Zi_denominator

        m    = 0.36 # Malis EELS paper - exponent m nach eq.(8)
        E_m  = 7.6*Z_eff**m # eq.(8): 7.6 eV
        
        E_0  = self.File.metadata.Acquisition_instrument.TEM.beam_energy
        beta = self.File.metadata.Acquisition_instrument.TEM.Detector.EELS.collection_angle

        F = (1 + E_0/1022)/(1 + E_0/511)**2 # Malis EELS paper - E_0 in keV nach eq.(6) 

        mean_free_path = 106 * F * E_0 / (E_m * np.log(2 * beta * E_0 / E_m))
        
        return mean_free_path
    
    
    # Calculate the parametershifts between the gausfits of neighbouring pixels
    def generate_linescans(self,
                           parameter_shifts = False,
                          ):
        """
        Comments missing
        """
        return
        
        
        
    def estimate_parametershift(self,
                                linescan,
                                maxpeakn = 1
                               ):
        """
        Calculate a parametershift by line scan analysis using a gaussian estimation
        for signifcant shifts and a linear estimation for thickness dependence.
        """
        #estimate thickness dependence of parameter shifts
        def func_lin(x, a, b):
            return a * x + b

        #estimate parameter shifts due to shear band influence
        def func_gaus(x, x0, height, sigma):
            return height * np.exp(- 1/(2 * sigma^2) * (x - x0)**2)

        #combined function to describe parameter shift 
        def func_all(x, a, b, x0, height, sigma):
            return a * x + b + height * np.exp(- 1/(2 * sigma^2) * (x - x0)**2)

        size                 = linescan.axes_manager.signal_axes[0].size
        scale                = linescan.axes_manager.signal_axes[0].scale
        offset               = linescan.axes_manager.signal_axes[0].offset
        max_pos              = offset + size * scale
        
        linescan_pos         = np.linspace(offset, max_pos, size)
        
        popt_lin, pcov_lin   = curve_fit(func_lin, linescan_pos, linescan.data)
        
        # generate the substracted array for peak finding.
        # look for peaks via ohaver, which only supports
        # finding positive peaks. Therefore check if peaks
        # were found and also check for the inverse.
        linear_substraction  = func_lin(linescan_pos, *popt_lin)
        substracted          = hs.signals.Signal1D(linescan.data - linear_substract)
        substracted_inv      = hs.signals.Signal1D(linear_substract - linescan.data)
        
        # set the signal axis for both signals 
        substracted.axes_manager.signal_axes[0]     = linescan.axes_manager.signal_axes[0]
        substracted_inv.axes_manager.signal_axes[0] = linescan.axes_manager.signal_axes[0]
        
        peaks_positive       = substracted.find_peaks1D_ohaver(maxpeakn = maxpeakn)
        peaks_negative       = substracted_inv.find_peaks1D_ohaver(maxpeakn = maxpeakn)
        print(peaks_positive, peaks_negative)
        
        popt_gaus, pcov_gaus = curve_fit(func_gaus, 
                                         linescan_pos, 
                                         linescan.data - linear_substract, 
                                         p0=peaks
                                        )

        popt_all, pcov_all   = curve_fit(func_all, 
                                         linescan_pos, 
                                         linescan.data,
                                         p0 = np.append(popt_lin, popt_gaus)
                                        )

        plt.xlabel('position [nm]')
        plt.ylabel('Plasmon energy [eV]')
        plt.title(r'SB1 - Al Y Fe')
        plt.plot(roi1_pos, 
                 func_all(roi1_pos, *popt_all), 
                 'r-', 
                 label='fit: a=%5.3f, b=%5.3f, A=%5.3f, B=%5.3f, x0=%5.3f' % tuple(popt_all)
                )
        plt.plot(roi1_pos, 
                 roi_1_centre.data
                )
        plt.legend()


        #mean_M = np.mean(func_lin(roi1_pos, *popt_all[:2]))
        #strain_energy[0,1] = func_all(popt_all[4], *popt_all)
        #shift_energy[0] = (func_all(popt_all[4], *popt_all) - mean_M) / mean_M
        #print('delta Ep: ' + str(shift_energy[0]) )

In [16]:
import cv2

img=roi_area.data
#img=cv2.imread('Figure_intensity_parameter_of_First_Plasmon_Peak_component_Signal.png') 

img=255*(1-(img-np.min(img))/(np.max(img)-np.min(img)))
#img=255*(img-np.min(img))/(np.max(img)-np.min(img))

img_gauss = cv2.GaussianBlur(img, (5,5), cv2.BORDER_DEFAULT)
img_gauss=img_gauss.astype('uint8')
vis = img_gauss.copy()
mser = cv2.MSER_create()
coordinates, bboxes = mser.detectRegions(img_gauss)

hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(img_gauss, hulls, 1, (0, 0, 255), 2)

FD = cv2.Feature2D('MSER')
kpoints = mser.detect(img_gauss)

mask = np.zeros((img.shape[0], img.shape[1], 1), dtype=np.uint8)

for contour in hulls:

    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1)
    
sb_only = cv2.bitwise_and(img_gauss, img_gauss, mask=mask)


coords = []
for coord in coordinates:
    bbox = cv2.boundingRect(coord)
    x,y,w,h = bbox
    if w< 10 or h < 10 or w/h > 5 or h/w > 5:
        continue
    coords.append(coord)

NameError: name 'roi_area' is not defined

In [13]:

"""
colors = [[43, 43, 200], 
          [43, 75, 200], 
          [43, 106, 200], 
          [43, 137, 200], 
          [43, 169, 200], 
          [43, 200, 195], 
          [43, 200, 163], 
          [43, 200, 132], 
          [43, 200, 101], 
          [43, 200, 69], 
          [54, 200, 43], 
          [85, 200, 43], 
          [116, 200, 43], 
          [148, 200, 43], 
          [179, 200, 43], 
          [200, 184, 43], 
          [200, 153, 43], 
          [200, 122, 43], 
          [200, 90, 43], 
          [200, 59, 43], 
          [200, 43, 64], 
          [200, 43, 95], 
          [200, 43, 127], 
          [200, 43, 158], 
          [200, 43, 190], 
          [174, 43, 200], 
          [142, 43, 200], 
          [111, 43, 200], 
          [80, 43, 200], 
          [43, 43, 200]
         ]
"""

## Fill with random colors
np.random.seed(0)
canvas1 = img_gauss.copy()
canvas2 = cv2.cvtColor(img_gauss, cv2.COLOR_GRAY2BGR)
canvas3 = np.zeros_like(canvas2)

for cnt in coords:
    xx = cnt[:,0]
    yy = cnt[:,1]
    color = colors[np.random.choice(len(colors))]
    #canvas1[yy, xx] = color
    canvas2[yy, xx] = color
    canvas3[yy, xx] = color

plt.imshow(canvas3)

NameError: name 'img_gauss' is not defined

In [145]:
for i in range(len(kpoints)):
    print('Coordinates: %f, %f' %(int(kpoints[i].pt[0]), int(kpoints[i].pt[1])))

Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 45.000000, 20.000000
Coordinates: 46.000000, 20.000000
Coordinates: 47.000000, 20.000000
Coordinates: 46.000000, 20.000000
Coordinates: 46.000000, 20.000000
Coordinates: 38.000000, 19.000000
Coordinates: 40.000000, 24.000000
Coordinates: 60.000000, 44.000000
Coordinates: 40.000000, 44.000000
Coordinates: 42.000000, 20.000000
Coordinates: 42.000000, 20.000000
Coordinates: 42.000000, 20.000000
Coordinates: 42.000000, 20.000000
Coordinates: 42.000000, 20.000000
Coordinates: 41.000000, 20.000000
Coordinates: 40.000000, 24.000000


In [146]:
#plt.imshow(vis)
cv2.imshow('img', vis)

#cv2.waitKey(0)

#cv2.destroyAllWindows()

In [43]:
regions[]

[array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [130,  23],
        [155,  15],
        [111,  17]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [131,  17],
        [141,  22],
        [112,  23]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [102,  19],
        [117,  17],
        [140,  22]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [ 94,  24],
        [134,  17],
        [133,  23]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [136,  22],
        [122,  22],
        [ 96,  20]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [127,  18],
        [112,  22],
        [115,  21]], dtype=int32), array([[  3,   3],
        [  4,   5],
        [  5,   5],
        ...,
        [114,  21],
        [100,  23],
     

# Testing _STZ-Mapping_ Routine

In [30]:
a.eels_fit_routine(function_set='VolumePlasmonDrude', fitter='Nelder-Mead', method='ml', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


params [array([(14.90902532, 112.05876665, 3.03228091),
       (29.9662939 ,  51.84620311, 6.0579164 )],
      dtype=[('position', '<f8'), ('height', '<f8'), ('width', '<f8')])]
fpp_fwhm 3.0322809136931803
fpp_int 52339.63976985681
spp_int 15701.891930957043
Estimate function parameters...


params [array([(14.90902532, 112.05876665, 3.03228091),
       (29.9662939 ,  51.84620311, 6.0579164 )],
      dtype=[('position', '<f8'), ('height', '<f8'), ('width', '<f8')])]
fpp_fwhm 3.0322809136931803
fpp_int 52339.63976985681
spp_int 15701.891930957043
52339.63976985681
15701.891930957043
Estimate function parameters...


params [array([(14.90902532, 112.05876665, 3.03228091),
       (29.9662939 ,  51.84620311, 6.0579164 )],
      dtype=[('position', '<f8'), ('height', '<f8'), ('width', '<f8')])]
fpp_fwhm 3.0322809136931803
fpp_int 52339.63976985681
spp_int 15701.891930957043
Estimate function parameters...


params [array([(14.90902532, 112.05876665, 3.03228091),
       (29.9662939 ,  51.84620311, 6.0579164 )],
      dtype=[('position', '<f8'), ('height', '<f8'), ('width', '<f8')])]
fpp_fwhm 3.0322809136931803
fpp_int 52339.63976985681
spp_int 15701.891930957043
52339.63976985681
15701.891930957043
Estimate function parameters...


params [array([(14.90902532, 112.05876665, 3.03228091),
       (29.9662939 ,  51.84620311, 6.0579164 )],
      dtype=[('position', '<f8'), ('height', '<f8'), ('width', '<f8')])]
fpp_fwhm 3.0322809136931803
fpp_int 52339.63976985681
spp_int 15701.891930957043
Estimate function parameters...


params [array([(14.90902532, 112.05876665, 3.03228091),
       (29.9662939 ,  51.84620311, 6.0579164 )],
      dtype=[('position', '<f8'), ('height', '<f8'), ('width', '<f8')])]
fpp_fwhm 3.0322809136931803
fpp_int 52339.63976985681
spp_int 15701.891930957043
52339.63976985681
15701.891930957043
52339.63976985681
15701.891930957043
Components	Parameter	Value
Zero_Loss_Peak
		FWHM	0.71102
		area	8.6817e+06
		centre	-0.0673084
		gamma	0.242764
First_Plasmon_Peak
		fwhm	2.71884
		intensity	175884
		plasmon_energy	15.0938
Second_Plasmon_Peak
		fwhm	10.7918
		intensity	50085.1
Do you want to adjust the starting parameters? (y/n): n
Correcting poissonian noise for the gain factor of
the EELS - detector...
Including gain factor of detector in poissonian noise. Rescaling...
Generating_seeds for SAMFire...


Adj. fit goodness (reduced Chi squared):  0.9976874165368038
Standard deviation of adj. fit goodness:  0.03589496200840594
Model will be stored in file...
Stored models in file:
{'VolumePlasmonDrude_leastsq_ls': <EELSModel, title: AlYFe SB3>, 'VolumePlasmonDrude_Nelder-Mead_ml': <EELSModel, title: AlYFe SB3>}


In [31]:
a.Fit_Model.plot(plot_components=True)
a.Fit_Model.red_chisq.plot()
print(np.mean(a.Fit_Model.red_chisq.data))
print(np.std(a.Fit_Model.red_chisq.data))
#a.plot_parameter_maps(False)

0.9976874165368038
0.03589496200840594


In [27]:
a=None
a=Plasmon_mapper(file=hs.load('../Master/EELS AlYFe 3 - 20190426/EELS Spectrum Image.dm3').inav[:20,:20], is_lazy=True)
a.File.metadata.General.title = 'AlYFe SB3'
#a = STZ_mapper(filename = '18_01_26_Spectrum_image_14Plasmon_FWHM.dm3', use_roi=False)

#m = a.eels_fit_routine()
#m.plot()
#m = im[0].create_model()
#m.print_current_values(only_free=False)
#print(m.red_chisq.data)
#print(m.components)
#m.components.Zero_Loss_Peak.function(x=-0.270658)

Does the file include stored models from previous fitting? (y/n): n
Setting up proper navigation space...
Aligning Zero-Loss Peak...

Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-0.22
std:	0.0496

min:	-0.35
Q1:	-0.25
median:	-0.2
Q3:	-0.2
max:	-0.1


Aligning datastructure successful. Estimate poissonian noise...
Loading process completed. Energy channels cropped by ZLP-Alignment: 8[0.390625%]
Channels before: 2048
Channels after: 2040
Please check microscope and detector parameters,
as the metadata from gatan might be incorrectly loaded
Hyperspy.
Do you want to load the corresponding darkfield image?
(Warning: The dimensions have to match exactly for some functions!) (y/n): n


In [4]:
a.plot_file()

In [12]:
a.calc_thickness(['Cu','Zr'], [0.64,0.36])

In [13]:
a.thickness_map.plot()

In [17]:
a.rect_roi(a.FPP_Emax, 100, 100, 150, 150)

Do you want to use the interactive gui?
Adjusting position is possible using the interactive gui. (y/n): y


NameError: name 'area' is not defined

In [5]:
a=None
a=Plasmon_mapper(file=hs.load('AlYFe-SB3_cropped_all_models.hspy'))

Does the file include stored models from previous fitting? (y/n): y
Modelinformation of loaded file: 
├── Gaussian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:03:23
│   └── dimensions = (20, 20|2040)
├── Gaussian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:09:51
│   └── dimensions = (20, 20|2040)
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:08:34
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   

In [47]:
a.load_model()

Modelinformation of loaded file: 
├── Gaussian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:03:23
│   └── dimensions = (20, 20|2040)
├── Gaussian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:09:51
│   └── dimensions = (20, 20|2040)
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:08:34
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date 

AttributeError: 'Voigt' object has no attribute 'A'

In [6]:
a.Fit_Model.plot(plot_components=True)
a.Fit_Model.red_chisq.plot()
print(np.mean(a.Fit_Model.red_chisq.data))
print(np.std(a.Fit_Model.red_chisq.data))
#a.plot_parameter_maps(False)

0.9792480064330104
0.04423923945906048


In [92]:
a.eels_fit_routine(function_set='Lorentzian', fitter='mpfit', method='ls', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Model will be stored in file...


AttributeError: 'NoneType' object has no attribute 'store'

In [33]:
a.eels_fit_routine(function_set='Gaussian', fitter='mpfit', method='ls', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:60: RuntimeWarning: divide by zero encountered in true_divide
  z = (np.asarray(x) - center + 1j * gamma) / (sigma * math.sqrt(2))
/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:61: RuntimeWarning: invalid value encountered in true_divide
  V = wofz(z) / (math.sqrt(2 * np.pi) * sigma)


...

Estimate function parameters...


Do you want to adjust the starting parameters? (y/n): n
Generating seeds...


Including gain factor of detector in poissonian noise. Rescaling...
Generating seeds...


[[       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 1.0051496         nan 0.99181156        nan 0.99756515
         nan 1.0332889         nan 1.00745364        nan 0.99160393
         nan 1.00806421        nan 1.03623511        nan 1.04992222
         nan 1.016366  ]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 1.00907479        nan 0.97000498        nan 0.99739093
         nan 0.96765862        nan 0.99187144        nan 1.03139337
         nan 1.01691235        nan 1.00679021        nan 1.01091673
         nan 1.01209577]
 [       nan        nan        nan        nan        nan        nan
         nan    

Model will be stored in file...
Stored models in file:
└── Gaussian_mpfit_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Loss_Peak
    ├── date = 2019-07-11 08:52:58
    └── dimensions = (20, 20|2040)



In [68]:
a.eels_fit_routine(function_set='Voigt', fitter='mpfit', method='ls', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Do you want to adjust the starting parameters? (y/n): n
Generating seeds...


Including gain factor of detector in poissonian noise. Rescaling...
Generating seeds...


[[       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 0.98232701        nan 0.96684066        nan 0.97865599
         nan 1.04208872        nan 0.97934911        nan 0.99553165
         nan 1.01389262        nan 1.04234853        nan 1.06118743
         nan 1.04657585]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 0.96920321        nan 0.94496232        nan 0.96935147
         nan 0.95199671        nan 0.98990854        nan 1.04843162
         nan 1.01409099        nan 0.99968435        nan 1.0277257
         nan 1.02675032]
 [       nan        nan        nan        nan        nan        nan
         nan     

Model will be stored in file...
Stored models in file:
├── Gaussian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:03:23
│   └── dimensions = (20, 20|2040)
├── Gaussian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:09:51
│   └── dimensions = (20, 20|2040)
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:08:34
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Lo

In [38]:
a.eels_fit_routine(function_set='VolumePlasmonDrude', fitter='mpfit', method='ls', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


52339.63976985681
15701.891930957043
Estimate function parameters...


Estimate function parameters...


52339.63976985681
15701.891930957043
Estimate function parameters...


/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:60: RuntimeWarning: divide by zero encountered in true_divide
  z = (np.asarray(x) - center + 1j * gamma) / (sigma * math.sqrt(2))
/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/site-packages/hyperspy/_components/voigt.py:61: RuntimeWarning: invalid value encountered in true_divide
  V = wofz(z) / (math.sqrt(2 * np.pi) * sigma)


Estimate function parameters...


52339.63976985681
15701.891930957043
52339.63976985681
15701.891930957043
Do you want to adjust the starting parameters? (y/n): n
Correcting poissonian noise for the gain factor of
the EELS - detector by generating statistics on the
:reduced Chi-squared.
Including gain factor of detector in poissonian noise. Rescaling...
Generating_seeds for SAMFire...


Adj. fit goodness (reduced Chi squared):  0.9975623356990309
Standard deviation of adj. fit goodness:  0.03475654845041352
Model will be stored in file...
Stored models in file:
{'VolumePlasmonDrude_leastsq_ls': <EELSModel, title: AlYFe SB3>, 'VolumePlasmonDrude_Nelder-Mead_ml': <EELSModel, title: AlYFe SB3>, 'VolumePlasmonDrude_mpfit_ls': <EELSModel, title: AlYFe SB3>}


In [51]:
a.Fit_Model.plot(plot_components=True)
a.Fit_Model.red_chisq.plot()
print(np.mean(a.Fit_Model.red_chisq.data))
print(np.std(a.Fit_Model.red_chisq.data))
#a.plot_parameter_maps(False)

0.9977571673681785
0.03858396764401258


In [57]:
a.eels_fit_routine(function_set='Voigt', fitter='leastsq', method='ls', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Do you want to adjust the starting parameters? (y/n): n
Generating seeds...


Including gain factor of detector in poissonian noise. Rescaling...
Generating seeds...


[[       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 0.94310262        nan 0.92823464        nan 0.9395782
         nan 1.00047805        nan 0.94024377        nan 0.95578057
         nan 0.97340755        nan 1.00072759        nan 1.01881433
         nan 1.00478601]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 0.93050288        nan 0.90722993        nan 0.93064523
         nan 0.91398341        nan 0.95038142        nan 1.00652135
         nan 0.97359826        nan 0.95976702        nan 0.986616
         nan 0.98575223]
 [       nan        nan        nan        nan        nan        nan
         nan       

Model will be stored in file...
Stored models in file:
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 09:26:22
│   └── dimensions = (20, 20|2040)
├── Lorentzian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:54:51
│   └── dimensions = (20, 20|2040)
├── Voigt_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 09:29:35
│   └── dimensions = (20, 20|2040)
└── VolumePlasmonDrude_leastsq_ls
    ├── components
    │   ├── First_Plasmon_Peak
    │   ├── Second_Plasmon_Peak
    │   └── Zero_Los

In [66]:
a.eels_fit_routine(function_set='Gaussian', fitter='leastsq', method='ls', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Do you want to adjust the starting parameters? (y/n): n
Generating seeds...


Including gain factor of detector in poissonian noise. Rescaling...
Generating seeds...


[[       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 1.0051496         nan 0.99181156        nan 0.99756515
         nan 1.0332889         nan 1.00745364        nan 0.99160393
         nan 1.00806421        nan 1.03623511        nan 1.04992222
         nan 1.01636599]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 1.00907479        nan 0.97000498        nan 0.99739093
         nan 0.96765862        nan 0.99187144        nan 1.03139336
         nan 1.01691235        nan 1.00679021        nan 1.01091673
         nan 1.01209577]
 [       nan        nan        nan        nan        nan        nan
         nan    

Model will be stored in file...
Stored models in file:
├── Gaussian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:03:23
│   └── dimensions = (20, 20|2040)
├── Gaussian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:09:51
│   └── dimensions = (20, 20|2040)
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:08:34
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Lo

In [58]:
a.eels_fit_routine(function_set='Lorentzian', fitter='leastsq', method='ls', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Do you want to adjust the starting parameters? (y/n): n
Generating seeds for gain correction...


Including gain factor of detector in poissonian noise. Rescaling...


Adj. fit goodness (reduced Chi squared):  67.79318020832261
Standard deviation of adj. fit goodness:  114.63404532292836
Model will be stored in file...


AttributeError: 'Plasmon_mapper' object has no attribute 'Fit_model'

In [36]:
a.eels_fit_routine(function_set='VolumePlasmonDrude', fitter='leastsq', method='ls', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


52339.63976985681
15701.891930957043
Estimate function parameters...


Estimate function parameters...


52339.63976985681
15701.891930957043
Estimate function parameters...


Estimate function parameters...


52339.63976985681
15701.891930957043
52339.63976985681
15701.891930957043
Do you want to adjust the starting parameters? (y/n): n
Correcting poissonian noise for the gain factor of
the EELS - detector by generating statistics on the
:reduced Chi-squared.
Including gain factor of detector in poissonian noise. Rescaling...
Generating_seeds for SAMFire...


Adj. fit goodness (reduced Chi squared):  0.9975623355217919
Standard deviation of adj. fit goodness:  0.034756548623404764
Model will be stored in file...
Stored models in file:
{'VolumePlasmonDrude_leastsq_ls': <EELSModel, title: AlYFe SB3>}


In [8]:
a.Fit_Model.plot(plot_components=True)
a.Fit_Model.red_chisq.plot()
print(np.mean(a.Fit_Model.red_chisq.data))
print(np.std(a.Fit_Model.red_chisq.data))
#a.plot_parameter_maps(False)

1.0029401861058853
0.03464272060852398


In [65]:
a.eels_fit_routine(function_set='Lorentzian', fitter='Nelder-Mead', method='ml', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Do you want to adjust the starting parameters? (y/n): n
Generating seeds...


Including gain factor of detector in poissonian noise. Rescaling...
Generating seeds...


[[       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 1.0046401         nan 0.98668006        nan 1.00196479
         nan 1.06822166        nan 0.99958108        nan 1.01904849
         nan 1.03772977        nan 1.06780356        nan 1.08527075
         nan 1.07192915]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 0.99123841        nan 0.96193939        nan 0.98945703
         nan 0.97142527        nan 1.0122186         nan 1.07379041
         nan 1.03708316        nan 1.02092727        nan 1.05162456
         nan 1.04914275]
 [       nan        nan        nan        nan        nan        nan
         nan    

Model will be stored in file...
Stored models in file:
├── Gaussian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:03:23
│   └── dimensions = (20, 20|2040)
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:08:34
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 09:26:22
│   └── dimensions = (20, 20|2040)
├── Lorentzian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Lo

In [64]:
a.eels_fit_routine(function_set='Gaussian', fitter='Nelder-Mead', method='ml', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Do you want to adjust the starting parameters? (y/n): n
Generating seeds...


Including gain factor of detector in poissonian noise. Rescaling...
Generating seeds...


[[       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 1.03995683        nan 1.02797271        nan 1.01626699
         nan 1.04051374        nan 0.9957306         nan 0.96919001
         nan 0.97720302        nan 0.97947715        nan 0.96608335
         nan 0.94128106]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 1.0290088         nan 0.98151252        nan 1.01421247
         nan 0.97907638        nan 0.97197584        nan 1.00037852
         nan 0.97181268        nan 0.96203742        nan 0.94379208
         nan 0.94492771]
 [       nan        nan        nan        nan        nan        nan
         nan    

Model will be stored in file...
Stored models in file:
├── Gaussian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:03:23
│   └── dimensions = (20, 20|2040)
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 09:39:21
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 09:26:22
│   └── dimensions = (20, 20|2040)
├── Lorentzian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Lo

In [61]:
a.eels_fit_routine(function_set='Voigt', fitter='Nelder-Mead', method='ml', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


Estimate function parameters...


...

Do you want to adjust the starting parameters? (y/n): n
Generating seeds...


Including gain factor of detector in poissonian noise. Rescaling...
Generating seeds...


[[       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 1.00482932        nan 0.98678107        nan 1.00210161
         nan 1.06821698        nan 0.99961333        nan 1.01902027
         nan 1.03772087        nan 1.06776703        nan 1.08520629
         nan 1.07188776]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan 0.99151447        nan 0.96203887        nan 0.9894643
         nan 0.9714545         nan 1.0122833         nan 1.07380427
         nan 1.03710587        nan 1.02088659        nan 1.05162097
         nan 1.04908019]
 [       nan        nan        nan        nan        nan        nan
         nan     

Model will be stored in file...
Stored models in file:
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 09:39:21
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 09:26:22
│   └── dimensions = (20, 20|2040)
├── Lorentzian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:54:51
│   └── dimensions = (20, 20|2040)
├── Voigt_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_

Process ForkPoolWorker-51:
Process ForkPoolWorker-50:
Process ForkPoolWorker-49:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/max/Apps/anaconda3/envs/Hyperspy/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (Tr

In [37]:
a.eels_fit_routine(function_set='VolumePlasmonDrude', fitter='Nelder-Mead', method='ml', samfire=True)

Reinitialising poissonian noise estimation...
Estimate function parameters...


Estimate function parameters...


52339.63976985681
15701.891930957043
Estimate function parameters...


Estimate function parameters...


52339.63976985681
15701.891930957043
Estimate function parameters...


Estimate function parameters...


52339.63976985681
15701.891930957043
52339.63976985681
15701.891930957043
Do you want to adjust the starting parameters? (y/n): n
Correcting poissonian noise for the gain factor of
the EELS - detector by generating statistics on the
:reduced Chi-squared.
Including gain factor of detector in poissonian noise. Rescaling...
Generating_seeds for SAMFire...


Adj. fit goodness (reduced Chi squared):  0.9976874274182446
Standard deviation of adj. fit goodness:  0.03589496022426341
Model will be stored in file...
Stored models in file:
{'VolumePlasmonDrude_leastsq_ls': <EELSModel, title: AlYFe SB3>, 'VolumePlasmonDrude_Nelder-Mead_ml': <EELSModel, title: AlYFe SB3>}


In [40]:
a.load_model()

Modelinformation of loaded file: 
├── Gaussian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:03:23
│   └── dimensions = (20, 20|2040)
├── Gaussian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:09:51
│   └── dimensions = (20, 20|2040)
├── Gaussian_mpfit_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 08:52:58
│   └── dimensions = (20, 20|2040)
├── Lorentzian_NelderMead_ml
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date = 2019-07-11 10:08:34
│   └── dimensions = (20, 20|2040)
├── Lorentzian_leastsq_ls
│   ├── components
│   │   ├── First_Plasmon_Peak
│   │   ├── Second_Plasmon_Peak
│   │   └── Zero_Loss_Peak
│   ├── date 

In [41]:
a.Fit_Model.plot(plot_components=True)
a.Fit_Model.red_chisq.plot()
print(np.mean(a.Fit_Model.red_chisq.data))
print(np.std(a.Fit_Model.red_chisq.data))
#a.plot_parameter_maps(False)

0.9976874274182446
0.03589496022426341


In [42]:
a.save_models_to_file('AlYFe-SB3_cropped_all_models.hspy')

Overwrite 'AlYFe-SB3_cropped_all_models.hspy' (y/n)?
y


In [15]:
im=a.Fit_Model
im.plot()
roi=hs.roi.RectangularROI(left=0, right=5., top=0, bottom=5)
imr=roi.interactive(im)
imr.plot()

TypeError: __getitem__() got an unexpected keyword argument 'out'

In [57]:
imr.plot()

In [14]:
roi = hs.roi.RectangularROI(0,0,10,10)
im_roi = roi(a.File)
im_roi.plot()

In [97]:
a.thickness(['Al','Y','Fe'], [0.88,0.07,0.05])
a.thickness_map.plot()
#a.thickness_map.save(filename='thickness map', extension='pgf')
plt.savefig(fname='thickness map', dpi=300, format='pgf')
plt.close()

No information on used beam energy found in metadata! Using default (300 keV).
No information on detector collection angle found in metadata! Using default (10 mrad).


In [19]:
sympy.init_printing()
x, t = sympy.symbols('x t', real=True)

mu = sympy.symbols('mu', real=True)
sigma, a, b, lamb, nu = sympy.symbols('sigma a b lambda nu', positive=True)

def area(dist):
    return sympy.simplify(sympy.integrate(dist, (x, -sympy.oo, sympy.oo)))
def mean(dist):
    return area(dist*x)
def EX2(dist):
    return area(dist*x**2)
def variance(dist):
    return sympy.simplify(EX2(dist) - mean(dist)**2)
def mgf(dist):
    return sympy.simplify(area(dist*sympy.exp(x*t)))
def latex(result):
    return "$" + sympy.latex(result) + "$\n" 
def summarize(dist):
    print("Distribution: " + latex(dist))
    print("Area: " + latex(area(dist)))
    print("Mean: " + latex(mean(dist)))
    print("Variance: " + latex(variance(dist)))
    print("MGF: " + latex(mgf(dist)))
summarise = summarize  # alias
gamma = sympy.Piecewise(
    (0, x < 0),
    (b**a / sympy.gamma(a) * x**(a-1) * sympy.exp(-x*b), True)
)
summarize(gamma)